In [383]:
import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation
import subprocess

p = 'santa-2023/'
path = pd.read_csv(p + 'puzzles.csv')
info = pd.read_csv(p + 'puzzle_info.csv')
sub = pd.read_csv(p+'sample_submission.csv')
# sub = pd.read_csv('submission.csv')

info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')

In [384]:
def do_action(state: list[str], action_id: str, allowed_moves: dict[str, list[int]]):
    s=state.copy()
    if action_id[0]=="-":
        action = allowed_moves[action_id[1:]]
        for i,a in enumerate(action):
            s[a]=state[i]
    else:
        action = allowed_moves[action_id]
        for i,a in enumerate(action):
            s[i]=state[a]
    return s

def simulate(state: str, actions: str, allowed_moves: str):
    s=state.split(";")
    allowed=eval(allowed_moves)
    for a in actions.split("."):
        s = do_action(s, a, allowed)
    return ";".join(s)

def simulate_inverse(state: str, actions: str, allowed_moves: str):
    s=state.split(";")
    allowed=eval(allowed_moves)
    for a in reversed(actions.split(".")):
        if a[0]=="-":
            s = do_action(s, a[1:], allowed)
        else:
            s = do_action(s, f"-{a}", allowed)
    return ";".join(s)

In [385]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

# git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
# cd rubiks-cube-NxNxN-solver
# make init
# pip install kociemba

In [400]:
# N=2
# N=3
# N=4
N=5
# N=6
# N=7
# N=8
# N=9
# N=10
# N=19
# N=33

NN=N*N

In [401]:
# org = "ABCDEF"
# import itertools
# org
# p = itertools.permutations(org)
# for a in p:
#     dct=dict(zip(a, "URFDLB"))
#     print(dct)
    
#     row=paths.loc[130]
#     id = row['id']
#     cur_state = row['initial_state']
#     print(cur_state)
#     input_state=state2ubl(cur_state)
#     print(id, input_state)

#     result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
#     output=result.stdout.decode()
#     if not output.startswith("Solution: "):
#         print("Fail!", output, result.stderr.decode())
#         continue
#     sol=output.replace("Solution: ","").replace("\n","")
#     print(sol)
#     print("find!!")
#     break

In [402]:
# "ACBDEF"
# "URFDLB"
# s="ABCDEF"
# s[0] + s[2] + s[1] + s[5] + s[4] + s[3]
dct=dict(zip("ACBFED", "URFDLB"))
dct

{'A': 'U', 'C': 'R', 'B': 'F', 'F': 'D', 'E': 'L', 'D': 'B'}

In [403]:
def state2ubl(state):

    # dct = {}
    # for i,u in enumerate(U):
    #     dct[state.split(';')[4+i*NN]] = u
    # print(dct)
    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:NN] + s[2*NN:3*NN] + s[NN:2*NN] + s[5*NN:] + s[4*NN:5*NN] + s[3*NN:4*NN]

In [404]:
eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0]).keys()

dict_keys(['f0', 'f1', 'f2', 'f3', 'f4', 'r0', 'r1', 'r2', 'r3', 'r4', 'd0', 'd1', 'd2', 'd3', 'd4'])

In [405]:
moves = eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

# TODO: 本質的に使わない操作が存在する？
# 他を回すと自分を固定する
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master/rubikscubennnsolver
M = {}
M["U"] = f'-d{N-1}'
M["R"] = "r0"
M["B"] = f"-f{N-1}"
M["F"] = "f0"
M["L"] = f"-r{N-1}"
M["D"] = "d0"
# if N>=4:
M["Uw"] = f'-d{N-1}.-d{N-2}'
M["Rw"] = "r0.r1"
M["Bw"] = f"-f{N-1}.-f{N-2}"
M["Fw"] = "f0.f1"
M["Lw"] = f"-r{N-1}.-r{N-2}"
M["Dw"] = "d0.d1"
# if N>=6:
#     M["2Uw"] = f'-d{N-1}.-d{N-2}'
#     M["2Rw"] = "r0.r1"
#     M["2Bw"] = f"-f{N-1}.-f{N-2}"
#     M["2Fw"] = "f0.f1"
#     M["2Lw"] = f"-r{N-1}.-r{N-2}"
#     M["2Dw"] = "d0.d1"
#     M["3Uw"] = f'-d{N-1}.-d{N-2}.-d{N-3}'
#     M["3Rw"] = "r0.r1.r2"
#     M["3Bw"] = f"-f{N-1}.-f{N-2}.-f{N-3}"
#     M["3Fw"] = "f0.f1.f2"
#     M["3Lw"] = f"-r{N-1}.-r{N-2}.-r{N-3}"
#     M["3Dw"] = "d0.d1.d2"
for i in range(2, N):
    M[f"{i}Uw"] = ".".join([f"-d{N-1-k}" for k in range(i)])
    M[f"{i}Rw"] = ".".join([f"r{k}" for k in range(i)])
    M[f"{i}Bw"] = ".".join([f"-f{N-1-k}" for k in range(i)])
    M[f"{i}Fw"] = ".".join([f"f{k}" for k in range(i)])
    M[f"{i}Lw"] = ".".join([f"-r{N-1-k}" for k in range(i)])
    M[f"{i}Dw"] = ".".join([f"d{k}" for k in range(i)])
    
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        # M[m+"'"] = "-"+M[m]
        M[m+"'"] = "-"+".-".join(M[m].split("."))
M

{'U': '-d4',
 'R': 'r0',
 'B': '-f4',
 'F': 'f0',
 'L': '-r4',
 'D': 'd0',
 'Uw': '-d4.-d3',
 'Rw': 'r0.r1',
 'Bw': '-f4.-f3',
 'Fw': 'f0.f1',
 'Lw': '-r4.-r3',
 'Dw': 'd0.d1',
 '2Uw': '-d4.-d3',
 '2Rw': 'r0.r1',
 '2Bw': '-f4.-f3',
 '2Fw': 'f0.f1',
 '2Lw': '-r4.-r3',
 '2Dw': 'd0.d1',
 '3Uw': '-d4.-d3.-d2',
 '3Rw': 'r0.r1.r2',
 '3Bw': '-f4.-f3.-f2',
 '3Fw': 'f0.f1.f2',
 '3Lw': '-r4.-r3.-r2',
 '3Dw': 'd0.d1.d2',
 '4Uw': '-d4.-d3.-d2.-d1',
 '4Rw': 'r0.r1.r2.r3',
 '4Bw': '-f4.-f3.-f2.-f1',
 '4Fw': 'f0.f1.f2.f3',
 '4Lw': '-r4.-r3.-r2.-r1',
 '4Dw': 'd0.d1.d2.d3',
 'U2': '-d4.-d4',
 "U'": 'd4',
 'R2': 'r0.r0',
 "R'": '-r0',
 'B2': '-f4.-f4',
 "B'": 'f4',
 'F2': 'f0.f0',
 "F'": '-f0',
 'L2': '-r4.-r4',
 "L'": 'r4',
 'D2': 'd0.d0',
 "D'": '-d0',
 'Uw2': '-d4.-d3.-d4.-d3',
 "Uw'": 'd4.d3',
 'Rw2': 'r0.r1.r0.r1',
 "Rw'": '-r0.-r1',
 'Bw2': '-f4.-f3.-f4.-f3',
 "Bw'": 'f4.f3',
 'Fw2': 'f0.f1.f0.f1',
 "Fw'": '-f0.-f1',
 'Lw2': '-r4.-r3.-r4.-r3',
 "Lw'": 'r4.r3',
 'Dw2': 'd0.d1.d0.d1',
 "Dw'": '-d0.-

In [406]:
# paths.loc[paths['puzzle_type'] == 'cube_3/3/3'].head()
# paths.loc[paths['puzzle_type'] == 'cube_4/4/4'].head()
# paths.loc[paths['puzzle_type'] == 'cube_5/5/5'].head()
# paths.loc[paths['puzzle_type'] == 'cube_6/6/6'].head()
# paths.loc[paths['puzzle_type'] == 'cube_7/7/7'].head()
# paths.loc[paths['puzzle_type'] == 'cube_33/33/33'].head()

In [407]:
# init →（M）→  ストライプ
# init →     → 普通の完成状態

In [394]:
input_state=simulate_inverse(
    # paths.loc[30, "solution_state"], ## 普通の解
    # paths.loc[130, "moves"],
    # paths.loc[130, "allowed_moves"],
    # paths.loc[150, "solution_state"], ## 普通の解
    # paths.loc[205, "moves"],
    # paths.loc[205, "allowed_moves"],
    paths.loc[210, "solution_state"], ## 普通の解
    paths.loc[240, "moves"],
    paths.loc[240, "allowed_moves"],
)
# paths.loc[130, "initial_state"]
input_state=state2ubl(input_state)
input_state

'BDBDFRURRDBUUULBDBLFDBLRRUDDDLUUUBLFBLFBUDFDRFBDUBBDLDLFRUFFFFRFFDDURULLRDFUBRBRRFLDLFLDFBFLUBBFUBBBRRLRDDUDURFULLBRRRBBDBLRDULUDDFRFUFLRUULLFFLRRULLB'

In [395]:
i=0

row=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[i]
id = row['id']
cur_state = row['initial_state']
print(cur_state)
input_state=state2ubl(cur_state)
print(id, input_state)

B;A;A;A;C;B;C;F;A;C;A;D;F;D;E;A;D;C;A;B;F;A;B;C;A;B;F;C;C;D;A;F;C;F;E;C;A;A;E;C;B;E;E;B;C;A;D;E;D;D;B;E;B;D;A;D;E;F;C;B;A;C;B;D;F;F;D;E;E;F;D;C;F;D;C;E;A;F;B;B;F;E;C;D;F;B;E;B;E;D;F;F;E;E;D;B
150 FUUURFRDURUBDBLURDLRUULRFLLFRUBLBRUFDUFRUFDRRBUDDLRBDFLFLBDDLLBFDDBLLDBRDBRLUDFFBBFLFBUBLDRFURFB


In [653]:
# input_state=state2ubl(paths.loc[130, "solution_state"])
# input_state

In [262]:
result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
output=result.stdout.decode()
if not output.startswith("Solution: "):
    print("Fail!", output, result.stderr.decode())
sol=output.replace("Solution: ","").replace("\n","")
sol

"D2 Rw2 Fw2 U2 Uw Fw Dw' B' Uw' Dw' Fw' D Fw2 Dw2 Rw' B F' Lw' U' Fw2 Lw' Fw2 D2 F U' R' Dw2 Lw2 U R' F D Lw2 Uw2 Lw2 Uw2 D2 F' D2 L2 Dw2 B L2 Uw2 Lw2 B D Rw2 D2 L2 Bw2 Lw2 F2 R2 U' Lw2 U2 Rw2 D' B2 Lw2 Fw2 U' D L' F2 D B' F L' R U B' F2 D' F2 U B2 U2 L2 U' F2 D'"

In [263]:
mmoves = '.'.join([M[m] for m in sol.split(' ')])
mmoves

'd0.d0.r0.r1.r0.r1.f0.f1.f0.f1.-d4.-d4.-d4.-d3.f0.f1.-d0.-d1.f4.d4.d3.-d0.-d1.-f0.-f1.d0.f0.f1.f0.f1.d0.d1.d0.d1.-r0.-r1.-f4.-f0.r4.r3.d4.f0.f1.f0.f1.r4.r3.f0.f1.f0.f1.d0.d0.f0.d4.-r0.d0.d1.d0.d1.-r4.-r3.-r4.-r3.-d4.-r0.f0.d0.-r4.-r3.-r4.-r3.-d4.-d3.-d4.-d3.-r4.-r3.-r4.-r3.-d4.-d3.-d4.-d3.d0.d0.-f0.d0.d0.-r4.-r4.d0.d1.d0.d1.-f4.-r4.-r4.-d4.-d3.-d4.-d3.-r4.-r3.-r4.-r3.-f4.d0.r0.r1.r0.r1.d0.d0.-r4.-r4.-f4.-f3.-f4.-f3.-r4.-r3.-r4.-r3.f0.f0.r0.r0.d4.-r4.-r3.-r4.-r3.-d4.-d4.r0.r1.r0.r1.-d0.-f4.-f4.-r4.-r3.-r4.-r3.f0.f1.f0.f1.d4.d0.r4.f0.f0.d0.f4.f0.r4.r0.-d4.f4.f0.f0.-d0.f0.f0.-d4.-f4.-f4.-d4.-d4.-r4.-r4.d4.f0.f0.-d0'

In [265]:
simulate(
    # paths.loc[130, "initial_state"],
    # mmoves,
    # paths.loc[130, "allowed_moves"],
    paths.loc[240, "initial_state"],
    mmoves,
    paths.loc[240, "allowed_moves"],
)

'N24;N23;N22;N21;N20;N19;N8;N7;N16;N15;N14;N11;N12;N13;N10;N9;N6;N17;N18;N5;N4;N3;N2;N1;N0;N75;N76;N77;N78;N79;N80;N81;N86;N83;N84;N85;N88;N87;N92;N89;N90;N93;N82;N91;N94;N95;N96;N97;N98;N99;N100;N101;N102;N103;N104;N105;N106;N117;N116;N109;N110;N111;N112;N113;N114;N115;N108;N107;N118;N119;N120;N121;N122;N123;N124;N25;N26;N27;N28;N29;N30;N41;N38;N33;N34;N35;N36;N37;N32;N39;N40;N43;N42;N31;N44;N45;N46;N47;N48;N49;N50;N51;N52;N53;N54;N55;N58;N63;N66;N59;N60;N57;N62;N61;N64;N65;N56;N67;N68;N69;N70;N71;N72;N73;N74;N149;N148;N147;N146;N145;N144;N133;N138;N131;N140;N139;N132;N137;N142;N135;N134;N143;N136;N141;N130;N129;N128;N127;N126;N125'

In [278]:
# I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
# for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
t=simulate(
    simulate_inverse(
        # paths.loc[240, "initial_state"],
        paths.loc[210, "solution_state"],
        mmoves,
        # paths.loc[240, "moves"],
        paths.loc[240, "allowed_moves"],    
    ),
    # mmoves+("."+init_moves if init_moves else ""),
    # mmoves,
    paths.loc[240, "moves"],
    paths.loc[240, "allowed_moves"],    
)
print(t)

A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F


In [ ]:
simulate(
    # paths.loc[130, "initial_state"],
    # mmoves,
    # paths.loc[130, "allowed_moves"],
    paths.loc[210, "initial_state"],
    mmoves,
    paths.loc[240, "allowed_moves"],
)

In [228]:
I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
    # if simulate(
    if simulate_inverse(
        paths.loc[130, "initial_state"],
        mmoves+("."+init_moves if init_moves else ""),
        paths.loc[130, "allowed_moves"],
    )==paths.loc[130, "solution_state"]:
        print(mmoves+"."+init_moves)

-f2.-f2.d0.d0.-r0.f2.-d0.r2.-d2.-d2.-f0.d0.d0.r0.d2.f0.f0.-d2.-d2.-r2.-r2.-d2.-d2.d0.d0.f2.-d2.-d2.-f2.-f2.-d2.-d2.f0.f1.f2.r0.r1.r2
-f2.-f2.d0.d0.-r0.f2.-d0.r2.-d2.-d2.-f0.d0.d0.r0.d2.f0.f0.-d2.-d2.-r2.-r2.-d2.-d2.d0.d0.f2.-d2.-d2.-f2.-f2.-d2.-d2.r0.r1.r2.d0.d1.d2.d0.d1.d2.d0.d1.d2
-f2.-f2.d0.d0.-r0.f2.-d0.r2.-d2.-d2.-f0.d0.d0.r0.d2.f0.f0.-d2.-d2.-r2.-r2.-d2.-d2.d0.d0.f2.-d2.-d2.-f2.-f2.-d2.-d2.d0.d1.d2.d0.d1.d2.d0.d1.d2.f0.f1.f2
-f2.-f2.d0.d0.-r0.f2.-d0.r2.-d2.-d2.-f0.d0.d0.r0.d2.f0.f0.-d2.-d2.-r2.-r2.-d2.-d2.d0.d0.f2.-d2.-d2.-f2.-f2.-d2.-d2.d0.d1.d2.f0.f1.f2.r0.r1.r2.r0.r1.r2
-f2.-f2.d0.d0.-r0.f2.-d0.r2.-d2.-d2.-f0.d0.d0.r0.d2.f0.f0.-d2.-d2.-r2.-r2.-d2.-d2.d0.d0.f2.-d2.-d2.-f2.-f2.-d2.-d2.f0.f1.f2.f0.f1.f2.r0.r1.r2.d0.d1.d2


In [451]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver?tab=readme-ov-file
# sol="D U' B' L B' U2 R U' L F U L2 U2 R2 F2 U R2 B2 U F2"
# sol="D' L2 Bw2 Rw' D Fw' D' Rw2 U Bw2 D B L Dw2 Rw' L2 U' F D2 Dw2 Rw2 F' L2 D' Bw2 Lw2 D2 F2 R2 D' Lw2 U L D B2 L' D L B' D2 F' R2 B2 D R2 U2 R2 D2 B2 L2 D'"

In [452]:
new_state = cur_state
mmoves = '.'.join([M[m] for m in sol.split(' ')])

for move in mmoves.split('.'):
    new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))

# I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
    temp_state = new_state
    if len(init_moves) > 0:
        for move in init_moves.split('.'):
            temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

    if temp_state == row['solution_state']:
        print(f'solved id: {id}')
        if len(init_moves) > 0:
            mmoves += '.' + init_moves
        if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
            print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
            paths.iloc[id,7] = mmoves
        break


solved id: 277
improved: new length 15067 vs current length 24419


# まとめて解く

In [662]:
import subprocess

for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    id = row['id']
    cur_state = row['initial_state']
    problem=state2ubl(cur_state)
    print("-"*30)
    print(id, problem)

    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {problem}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
        continue
    sol=output.replace("Solution: ","").replace("\n","")
    print("solved", sol)

    new_state = cur_state
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    for move in mmoves.split('.'):
        new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    
    I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp_state = new_state
        if len(init_moves) > 0:
            for move in init_moves.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
    
        if temp_state == row['solution_state']:
            print(f'solved id: {id}')
            if len(init_moves) > 0:
                mmoves += '.' + init_moves
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves
            break


------------------------------
150 FUUURFRDURUBDBLURDLRUULRFLLFRUBLBRUFDUFRUFDRRBUDDLRBDFLFLBDDLLBFDDBLLDBRDBRLUDFFBBFLFBUBLDRFURFB
solved D' L2 Bw2 Rw' D Fw' D' Rw2 U Bw2 D B L Dw2 Rw' L2 U' F D2 Dw2 Rw2 F' L2 D' Bw2 Lw2 D2 F2 R2 D' Lw2 U L D B2 L' D L B' D2 F' R2 B2 D R2 U2 R2 D2 B2 L2 D'
solved id: 150
improved: new length 98 vs current length 700
------------------------------
151 DUURLBFBFBFLLUBFUBRFLRRULUDRDURRBBDRBUBFULDBDDFBBLDRFFDRDDRDFFDBLDUUFULRFRULUBLLDLBFFLBDULFRURRL
solved Rw' Bw Rw B2 Dw' R2 F' Dw2 Rw' B2 U Rw' U2 Rw' Uw2 F L2 U' Bw2 U' F Dw2 B2 Bw2 D' F2 Fw2 B2 Lw2 R2 D Rw2 L2 D' L F L2 F L' F B' R U2 B2 U L2 F2 D2 R2 F2 U L2
solved id: 151
improved: new length 102 vs current length 610
------------------------------
152 RRDBDUBLUDBLFDUUFFDURFLLDLUBLLFBRBLLRLDDRRFFUBUDLLFFFURDUDLULRRRFFRDUFBBBBRUBBLBRRBUURUBDFDLDFFD
solved Dw2 Rw' D R2 Uw R' Fw' F' B' Rw' Bw2 Rw' Dw2 B' L' U Rw' Rw2 D B' Uw2 D B Dw2 B2 L2 D2 R2 B2 D' F2 Fw2 U' B2 Lw2 U2 D R B' U' B' U D2 R2 F R U' R2 U 

KeyboardInterrupt: 

In [467]:
paths[["id", "moves"]].to_csv("solver_submit.csv", index=None)

# ストライプ

In [237]:
import subprocess

normal_state=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[0]["solution_state"]
for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    if row["solution_state"]==normal_state:
        continue
    id = row['id']
    cur_state = row['initial_state']
    print("-"*30)
    print(id)

    input_state=simulate_inverse(
        normal_state, ## 普通の解
        row["moves"],
        row["allowed_moves"],
    )
    input_state=state2ubl(input_state)
    
    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
    sol=output.replace("Solution: ","").replace("\n","")
    print("sol", sol)
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    cur_state=simulate(
        row["initial_state"],
        mmoves,
        row["allowed_moves"],
    )
    
    I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp = cur_state
        if init_moves:
            temp=simulate(
                temp,
                init_moves,
                row["allowed_moves"],
            )
        if temp==row["solution_state"] or (row["num_wildcards"]>0 and sum(i!=j for i,j in zip(row["solution_state"].split(";"), temp.split(";")))<=row["num_wildcards"]):
            print(mmoves+"."+init_moves)
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves+("."+init_moves if init_moves else "")
            print("use wildcard" if temp!=row["solution_state"] else "")
            break


------------------------------
200
sol B2 U Dw' Bw Dw2 Fw' F' R2 U Rw' F' B U' Dw2 Lw' Rw2 U' Fw2 U' F Uw2 Bw2 D L2 U Lw2 U L2 R2 Bw2 D' Lw2 D' L' D' R L B U B' D2 B' L' U' F2 D2 B2 R2 B2 D R2 U2
------------------------------
201
sol Bw' Dw2 F' L' Uw Fw' L' U F L' U2 Rw Bw2 Lw' B Rw2 U2 Fw2 D B Dw2 Bw2 L2 D Bw2 U' R2 Bw2 D F2 Rw2 L U' R' U L2 U' R F U R D F2 L2 U' F2 D2 R2 F2 D2 R2
------------------------------
202
sol Rw' Dw' R2 U Uw' Bw' Dw2 R2 B L Rw' F' L' Dw2 Rw' B D Rw2 Bw2 D2 B Uw2 L2 Lw2 D' B2 Lw2 D L2 U Fw2 D' Lw2 U R2 B2 U' B2 L' D B' U' R B L2 U F2 D B2 D L2 U' L2 U'
------------------------------
203
sol B' U' Rw Dw Bw' F L U' Dw2 B' Rw B2 Bw2 Lw' U L2 B D F Dw2 Lw2 D' Bw2 U2 L2 D R2 Bw2 R2 Bw2 U' L D' B' R D' B2 U L F' D B2 R2 D2 R2 U' L2 U B2 D2
------------------------------
204
sol Bw D2 L' Rw Uw' D2 B Rw' U B Rw' Dw2 R' Rw2 U2 R2 B Bw2 Uw2 D F2 U F2 U Bw2 U' R2 Bw2 L' F2 R2 F L D L U D2 R B R2 U B2 U' R2 D' F2 L2 B2 R2
------------------------------
205
sol Bw2 L Dw2

In [154]:
paths[["id", "moves"]].to_csv("solver_submit_stripe.csv", index=None)

In [155]:
for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    assert simulate(
        row["initial_state"],
        row["moves"],
        row["allowed_moves"]
    )==row["solution_state"]

In [156]:
sol283="16Fw' 16Uw' L2 F 16Rw F 16Bw 16Dw2 16Rw 16Bw' F' 16Fw2 15Bw 16Dw' 15Rw 16Bw2 15Dw' 16Uw R' 16Uw' D 15Fw 15Dw2 15Rw2 D 15Uw 15Lw 15Bw' 15Rw' R' 14Rw2 16Uw 16Bw2 16Rw2 D' 16Lw2 B' 16Uw 16Lw2 F 14Bw 16Dw' 15Bw2 L' 15Lw2 15Dw 14Bw' R' 15Dw' 15Rw2 14Uw2 B2 14Bw' 15Dw' 14Rw2 14Fw B2 14Rw' 14Uw 14Bw' 14Lw' 13Rw D' 16Fw2 U' 16Bw2 13Lw' 16Dw 16Uw 13Bw' 16Lw2 13Rw2 U2 16Fw2 L2 15Fw2 15Dw 13Lw' 15Fw2 R 15Bw2 15Uw 13Rw' 13Fw 15Uw' 14Rw' F2 14Bw2 14Dw' F2 14Bw2 R 14Dw 14Rw' D' 13Fw 14Dw 14Uw' 13Fw 14Dw' 13Uw2 14Fw2 14Dw' L' 13Rw 14Dw R2 13Rw 13Fw' 13Dw' F' 13Rw' F' 12Dw' 16Bw2 16Uw2 12Rw' 16Dw' 16Fw2 16Bw2 16Dw 12Uw 12Fw' 16Dw 15Dw' 15Fw2 15Lw2 12Dw' 15Fw2 12Rw 15Uw2 12Fw' 15Dw 15Fw2 L 12Lw' 15Uw 14Uw' 14Rw2 U' 14Fw2 14Rw2 12Bw' D 14Bw2 14Dw2 L2 U 12Uw' 14Bw2 13Fw' D B2 13Uw' 13Fw' 13Bw2 13Dw' 13Uw 13Fw' U2 13Bw2 13Dw2 13Rw2 13Uw 13Lw2 13Fw2 L' 13Lw2 13Dw2 B 12Bw 13Uw2 12Bw D' R' 12Fw 12Bw' 12Lw 12Bw' 11Rw' 16Rw2 16Dw2 16Rw2 D2 16Lw2 16Fw2 11Rw' 16Bw2 16Dw F B2 16Uw2 11Fw' D 15Rw2 15Bw2 15Rw2 11Bw' 15Dw' L' 11Rw F 15Dw 14Uw2 11Bw' 14Uw B 14Dw' 14Bw2 14Dw 14Lw2 14Fw2 11Rw2 11Uw 14Rw2 13Bw2 13Rw2 13Uw' 13Fw2 13Uw2 11Rw2 11Bw' 13Uw 13Lw2 R 11Bw 13Dw2 12Uw2 12Lw 12Dw' 12Rw2 D' 12Bw' 12Dw 12Bw 12Lw' 11Rw' 12Uw2 12Fw2 12Bw2 12Lw2 U2 11Bw' 12Lw2 D2 12Rw2 F2 R 11Rw 12Uw' 11Bw' 11Uw 11Rw2 11Dw2 11Lw' 11Bw' R' 16Uw2 L2 16Lw2 16Bw2 16Uw 10Lw' 16Dw 16Fw2 10Rw' F' 10Bw' 16Dw' B 10Lw' 15Dw 15Rw2 15Fw2 15Bw2 10Rw2 10Bw 15Uw' 10Rw' 10Fw R' 15Dw' 14Uw' 14Lw2 14Bw2 L 14Dw2 10Fw' 10Lw 14Fw2 14Uw 10Rw2 10Bw' R 14Fw2 13Rw2 10Bw' 13Bw2 13Uw 13Fw2 L 13Dw2 10Bw' 13Uw2 10Bw 10Rw' 10Fw 13Uw R' 10Bw2 12Uw2 10Fw 12Uw2 B' 12Dw' 12Lw2 B2 10Bw' 12Dw' 12Fw2 11Rw D 11Fw 11Uw 11Fw L 11Dw 11Bw2 11Lw' 11Rw2 11Dw2 10Rw' 11Dw' 11Fw2 11Dw2 11Lw2 11Uw' 10Rw2 10Uw' F' 11Rw2 10Fw2 D' 10Lw' F 10Bw' 10Rw' 10Bw' 9Bw' 16Lw2 D2 B' 16Uw2 16Bw2 9Rw' 16Dw R2 16Bw2 9Rw2 9Bw' 16Uw' 15Uw' B' 15Bw2 15Lw2 R 15Bw2 9Uw2 9Lw' 15Dw 9Rw2 14Uw 14Bw2 14Lw2 14Rw2 9Bw 14Uw F' 14Lw2 14Rw2 14Uw2 9Bw 9Uw' 14Fw2 13Rw2 13Uw2 13Lw2 9Rw' 13Bw2 R' 13Dw' 9Bw' L 9Fw' 13Dw' 12Bw2 9Rw 12Uw' 12Bw2 D 12Rw2 9Bw' 9Lw F2 12Dw' F' 11Fw2 11Dw' 11Lw2 R' 9Bw 11Uw' 9Lw' D' 9Bw' 11Dw' 10Rw U 10Lw2 R 10Rw2 10Uw' 10Rw2 10Uw 10Rw' R 10Rw2 10Uw 10Bw2 10Dw2 10Rw2 D' 10Fw2 9Rw' 10Uw' 9Bw' L' 10Dw' 9Rw 9Fw' 9Bw' 9Dw' 9Lw' B' 9Rw' 16Bw2 16Dw2 16Uw 8Lw' 16Bw2 16Lw2 D 16Rw2 D2 8Rw' 16Dw' 15Dw 8Rw' 15Rw2 15Bw2 8Rw2 B' 15Dw2 15Uw' 8Rw U2 8Bw' 15Rw2 15Uw2 R 14Rw2 14Fw2 14Bw2 8Rw' 8Bw 14Uw 14Fw2 U 8Uw 8Bw' 14Dw2 13Rw2 8Bw2 R2 13Fw2 13Uw 13Rw2 8Bw 13Dw2 13Fw2 R 13Bw2 U' 13Uw 13Rw2 8Bw' 12Dw' 12Uw2 8Fw2 B 12Uw' 12Rw2 D' 8Bw2 8Uw 12Bw2 8Bw' 11Dw2 L 8Rw 11Uw 11Bw2 11Dw' 11Rw2 11Uw2 11Bw2 R2 8Fw 11Dw' 10Fw2 10Uw2 8Lw 10Dw' F2 8Fw' 10Bw2 10Dw 10Bw2 8Dw' 8Bw 10Dw' 9Rw' D 9Dw2 9Uw2 9Rw 9Dw 9Uw2 9Rw' 9Rw2 9Bw2 9Uw2 8Rw 9Dw2 9Bw2 D2 9Fw2 B2 9Dw2 F' 8Bw' 9Uw' U 8Fw L2 8Rw' U2 8Fw' 8Lw' 7Bw 16Uw 16Lw2 R' 16Uw2 L 16Dw' 16Lw2 F 16Dw L2 7Lw 16Uw 7Rw B2 15Uw' 7Bw' 15Dw2 15Rw2 7Fw2 7Lw' 15Uw 7Bw' 15Lw2 7Rw' 7Bw 14Dw2 R' 14Fw2 7Rw 14Dw 7Rw 7Bw 14Dw2 14Uw' 7Uw2 7Fw 13Bw2 13Dw' 13Uw 7Fw' 13Lw2 13Fw2 7Rw' B2 7Bw2 13Dw' 7Dw2 7Lw' 12Dw2 12Uw 12Bw2 7Uw' 12Fw2 R2 12Bw2 L' 7Bw 7Uw' 12Lw2 11Bw2 U' 11Fw2 11Uw2 7Rw 11Uw 7Fw' 11Rw2 11Dw' 11Bw2 7Dw B 11Dw 7Lw2 10Uw L' 10Fw2 10Dw L2 10Bw2 7Uw 7Bw 7Rw' 10Uw2 9Fw2 9Rw2 B2 9Dw2 7Bw 9Uw' 7Rw' 9Dw U 9Rw2 U2 9Rw2 8Rw' 8Uw 8Lw R2 8Uw' 8Rw 8Bw2 8Uw B2 8Lw' 7Fw2 8Dw2 8Rw2 7Bw 8Dw' R2 8Bw2 U' 8Lw2 7Rw2 F' 7Bw 8Uw' 7Rw2 7Fw' U' B 7Uw' 7Lw' 7Bw' 16Uw' 16Bw2 16Uw 6Bw' 6Lw' 16Uw L 16Dw2 6Rw F 6Lw' 16Dw' 15Rw2 15Uw' 6Fw 15Lw2 15Uw' 15Lw2 15Uw' 15Bw2 D 15Rw2 6Uw' 15Lw2 15Uw2 14Uw' F' 14Lw2 14Uw2 6Fw' D' 14Bw2 6Uw2 6Bw D' 14Lw2 13Rw2 13Dw' 6Bw 6Rw' 13Dw' 13Uw' 13Lw2 D2 13Fw2 D2 6Uw' 6Rw' 13Dw 12Bw2 12Uw' 12Bw2 6Dw' 12Fw2 U' 12Bw2 6Uw' 6Bw' 12Lw2 D 6Uw2 12Lw2 6Bw' 11Bw2 11Dw' 6Uw2 6Bw' 11Dw' R 6Uw B' 11Rw2 10Uw' B' 6Lw 10Dw2 10Lw2 10Dw 10Rw2 10Dw' B 6Lw' U' 10Rw2 6Rw' 6Bw 9Rw2 D2 9Lw2 9Fw2 9Dw2 9Uw 6Lw' 9Uw 6Rw' 9Dw2 9Uw B' 8Dw' 8Uw' 6Bw 8Dw' 8Bw2 6Dw2 8Lw2 8Bw2 U' 6Rw' 8Uw2 7Rw 7Dw 7Bw2 7Dw' U2 7Lw 7Uw 7Fw2 7Lw' 7Dw2 7Rw2 B' 7Lw2 7Fw2 L' 6Bw 7Uw 6Lw' F2 6Uw 7Lw2 6Rw2 6Fw' 6Dw2 6Rw 6Dw 6Fw' 5Rw2 16Uw2 L' 16Fw2 D' 16Bw2 L 16Uw 16Fw2 16Lw2 5Fw 5Uw' B2 16Lw2 5Rw' 5Bw 15Rw2 15Dw' 15Bw2 15Dw2 15Rw2 5Bw' 5Rw2 15Uw' 5Bw U2 15Rw2 5Rw' 14Bw2 14Rw2 14Dw 14Fw2 R 14Uw2 5Fw' 14Rw2 B2 14Dw2 L 5Rw 14Fw2 5Rw 5Uw 13Uw2 13Bw2 13Dw 13Rw2 13Dw2 5Rw 5Bw2 R 13Bw2 12Rw2 12Dw' R 12Bw2 12Dw2 12Uw F' 12Lw2 12Uw' L 5Rw 12Bw2 5Rw2 F' 11Uw2 5Bw 11Rw2 5Bw2 11Dw' 5Lw 11Bw2 5Uw2 5Fw 11Dw 5Rw 10Dw' 10Lw2 10Fw2 10Dw2 10Lw2 10Fw2 B' 10Uw 10Rw2 5Uw2 5Fw 10Dw' 9Rw2 9Uw' 9Fw2 9Bw2 9Rw2 5Bw' 9Dw' 5Rw2 5Bw 9Dw2 5Rw 5Uw2 B 9Uw 8Dw 8Fw2 8Rw2 5Bw 8Lw2 5Rw 8Fw2 D' 8Uw 5Lw' 5Bw2 8Dw' B' 7Dw 7Lw2 7Uw 5Rw' 7Dw2 7Rw2 7Dw' 5Bw2 5Rw' 7Bw2 6Bw D2 U' B2 6Lw 6Dw' 6Lw' 6Uw 6Bw' 5Rw' 5Bw' 6Uw' 6Rw2 B 6Lw2 6Uw2 5Rw 6Dw' 5Uw2 5Rw U2 6Bw2 5Rw' 5Bw' U' 5Fw' 5Dw 5Lw' 16Lw2 16Dw' 4Rw' 16Fw2 16Uw 4Rw 4Bw 16Dw' 4Fw' 4Rw 16Dw 4Bw 15Rw2 15Uw2 4Fw 15Dw 15Fw2 4Uw' 4Lw 15Dw2 15Lw2 4Fw' R2 15Dw' 14Uw L 14Fw2 14Rw2 14Uw 14Lw2 14Uw' F 14Dw' 14Lw2 F' R' 14Uw 13Rw2 13Fw2 R' 13Bw2 13Uw' F' 13Dw2 13Bw2 4Uw' 4Fw' 4Dw F 13Uw 12Rw2 12Bw2 L' 4Rw' 12Dw' 12Uw L' 4Bw 12Dw 4Fw' U 12Rw2 11Dw' B 11Rw2 4Fw 11Fw2 11Dw2 11Bw2 11Dw' 11Fw2 L 4Rw 11Bw2 10Uw2 B2 4Lw 10Rw2 10Dw 10Uw2 10Lw2 U 10Rw2 F 4Lw 10Dw' 4Rw' 9Bw2 9Dw' R' 9Uw 4Rw' 9Dw' 4Bw 9Lw2 4Dw2 4Rw B' 9Uw 8Uw' R' 8Uw' 8Bw2 8Lw2 8Rw2 4Fw' 8Uw2 4Rw F 8Dw 4Rw 7Fw2 7Rw2 7Dw' U2 4Bw' 7Uw 4Rw' 7Dw2 F 7Uw 4Rw' 6Uw2 R 6Bw2 6Dw 6Rw2 6Uw2 4Bw' 6Bw2 6Lw2 4Rw' 6Bw2 5Rw' 5Bw2 5Dw B2 R 5Dw' 5Rw' 5Bw2 4Rw2 4Dw 5Bw2 5Dw2 F 5Lw2 5Uw 5Lw2 4Bw2 4Lw' B2 5Dw 4Fw D' 4Uw2 4Lw' B' 4Rw' 4Fw' 3Rw2 16Dw' 3Bw' 16Lw2 16Fw2 3Bw' 16Lw2 16Fw2 16Bw2 3Rw2 F' 3Bw2 16Dw' R' 3Fw' 15Dw 15Lw2 U' 15Bw2 3Lw' 15Uw2 R 3Bw 3Uw' 15Fw2 U' 14Lw2 14Rw2 14Fw2 L 14Fw2 3Lw 14Bw2 14Uw' B2 14Uw' L 14Uw 13Rw2 13Uw 13Bw2 R' 13Rw2 13Dw' 13Lw2 3Rw' 13Bw2 13Dw' 3Uw' 3Bw R 13Uw2 12Rw2 12Dw2 B2 12Dw B2 12Lw2 R2 12Bw2 3Lw' 3Fw' L' 12Uw2 3Rw' 11Fw2 11Uw 3Lw 11Bw2 11Dw2 R 11Rw2 11Uw 3Bw' 3Rw' B' 11Uw2 10Uw2 F' 10Dw2 10Rw2 10Dw2 10Fw2 3Rw' 10Dw 3Lw 10Uw 3Rw' B2 10Uw' 9Bw2 3Rw' U 9Fw2 9Uw' 9Fw2 9Dw 3Fw' 9Dw 9Fw2 L2 F' 9Dw' R2 3Dw 3Lw 8Dw2 8Fw2 D' 8Rw2 8Dw 8Uw 8Lw2 U 8Lw2 3Rw2 7Bw2 7Uw' 7Fw2 R' 3Rw' 7Uw2 7Rw2 3Bw' U2 3Rw' B2 7Uw R 6Dw' 3Fw 6Rw2 6Fw2 6Lw2 B2 6Dw2 3Rw 6Uw' L2 6Dw2 3Rw2 U' 5Uw' 5Lw2 5Uw2 5Fw2 5Dw 5Bw2 3Lw U' 3Bw' 5Dw 4Bw U 4Fw' R2 4Uw' 4Lw2 4Bw U 4Fw' 3Rw' 4Bw2 4Dw' 4Fw2 3Bw' R2 4Uw' 3Bw U2 3Bw' 4Dw B2 3Rw' D2 3Fw 3Dw' 3Bw' 3Lw' 2Dw2 2Fw' 16Uw2 2Rw 16Rw2 16Dw' 16Rw2 U' 16Fw2 B2 2Dw2 2Rw' 16Uw2 2Lw' 15Fw2 15Bw2 15Dw' 15Uw2 2Lw' 15Dw 15Bw2 2Lw2 15Dw' 2Rw2 2Fw 15Uw' 14Dw' 14Rw2 14Bw2 2Uw L 14Fw2 14Uw 2Rw2 2Uw2 14Rw2 2Bw 2Lw' 14Dw 2Bw' R2 13Fw2 13Dw2 B2 13Uw 2Lw 13Uw 13Fw2 D2 2Lw 2Uw 13Lw2 R' 12Dw 2Rw2 12Bw2 12Uw' 12Rw2 12Bw2 R' 12Uw 12Fw2 2Rw 2Bw2 L' 12Fw2 2Rw' U2 11Uw 11Fw2 2Uw' 11Lw2 R2 11Uw' 2Fw2 L B' 11Uw 10Lw2 10Dw 2Rw' 10Uw2 L2 2Dw 10Bw2 D 2Rw B 10Uw' 9Lw2 B 9Uw F 9Dw 2Rw2 U' 2Bw 2Lw 9Dw' 8Dw2 8Rw2 U 8Lw2 R' 8Dw' B2 8Rw2 8Dw' U2 2Lw 8Dw' 7Uw' 7Fw2 2Bw 7Uw' 7Lw2 2Rw 7Uw 2Rw 7Fw2 2Rw B' 7Rw2 R2 6Dw2 2Lw F 6Dw' 6Lw2 6Dw 6Fw2 2Rw 6Dw 2Bw2 6Uw' 6Fw2 R' 2Bw 5Dw' 5Rw2 5Uw2 R2 2Rw2 5Bw2 2Rw' 2Uw 2Rw' 5Dw2 2Uw2 F 4Lw2 R' 4Uw 2Lw' 4Dw R 4Uw F' 4Rw2 3Lw' 3Dw 3Uw2 3Bw2 3Rw' 3Fw2 U' 3Rw' B' 3Uw' 3Bw2 3Rw2 3Dw 3Lw2 F2 3Uw 3Bw2 2Rw2 2Fw' L' 3Bw2 D2 R' 2Fw 2Rw 2Fw B2 2Lw 2Dw' 2Fw' 16Uw 16Bw2 16Uw2 R 16Dw' 16Uw2 F' 16Rw2 16Uw' 16Lw2 16Fw2 L B' 16Dw2 B' 15Dw 16Bw2 L 16Uw2 15Uw B' 15Rw2 15Uw' 15Rw2 F' 15Uw' L 15Dw' 16Rw2 16Bw2 14Uw' F 16Dw2 L' 16Bw2 14Dw' 14Uw R 16Bw2 15Rw2 15Bw2 15Dw2 15Fw2 R 15Dw2 B 15Rw2 14Uw' 15Lw2 14Uw2 15Bw2 R' 14Fw2 14Dw' L F' B' 14Uw' 16Dw2 B' 16Rw2 16Bw2 16Uw2 R 16Bw2 13Dw' 13Uw2 B 16Lw2 15Rw2 15Dw2 15Fw2 L 15Uw2 R' 13Uw2 R 15Dw2 F 15Lw2 R' 14Dw' 14Lw2 F' 14Dw' F 14Uw2 14Rw2 14Dw' L' 14Fw2 13Uw2 14Rw2 13Uw2 L 14Bw2 13Uw 14Lw2 13Bw2 13Uw2 B' R' 13Dw' F 13Uw' 16Bw2 16Uw2 F' 16Dw2 F' 12Dw2 16Lw2 12Uw L' 16Rw2 16Bw2 16Rw2 15Rw2 12Uw2 15Bw2 L 15Fw2 L B' 15Dw2 R2 15Uw2 14Fw2 14Uw2 F' 12Uw R' 14Dw2 R2 12Uw F' 14Lw2 R 14Fw2 13Bw2 13Uw' 13Lw2 13Rw2 F 13Dw F B2 13Dw' 13Bw2 13Dw2 13Uw2 13Rw2 13Dw2 13Lw2 12Dw2 R 13Fw2 B 13Uw2 F' 12Uw' B 12Dw2 L 12Dw' R' 16Bw2 16Rw2 16Uw2 16Rw2 B 11Uw' 16Uw2 B 16Dw2 L 15Uw2 R' 15Dw2 15Lw2 11Uw' F' 11Uw 15Bw2 14Uw2 B2 14Dw2 14Bw2 11Dw L 14Fw2 14Rw2 F R' 14Dw2 13Lw2 R' 13Uw2 11Fw2 B' 13Uw2 R 13Rw2 13Bw2 11Dw' R 13Fw2 F2 12Uw' F2 12Fw2 12Uw' 12Rw2 11Uw2 F' 12Fw2 12Lw2 12Uw2 L' 12Dw2 L' 11Rw2 12Dw2 11Dw2 11Uw' F' R' 11Dw' 16Rw2 B' 10Dw' L 16Fw2 16Lw2 16Fw2 16Rw2 B 16Rw2 B' 15Rw2 10Uw' 15Fw2 10Lw2 15Uw2 R' 10Dw 15Dw2 B' 15Lw2 R 10Uw L' 14Rw2 14Fw2 10Rw2 B 14Dw2 10Uw' 14Lw2 B' 13Rw2 13Uw2 R 13Dw2 13Rw2 13Bw2 13Dw2 13Rw2 13Bw2 10Uw' 13Bw2 R' 12Uw2 12Rw2 F 12Uw2 L 12Dw2 L' 10Bw2 12Bw2 10Uw 12Fw2 11Dw' F 11Fw2 11Uw2 F 11Dw' 11Rw2 11Uw2 10Rw2 11Fw2 10Uw' 11Lw2 11Uw2 L' 11Bw2 L 10Dw' F' 11Dw2 R 10Dw' L' F' R' 10Dw' B 10Uw' 9Rw2 16Uw2 16Fw2 9Uw' F' 16Dw2 9Bw2 R' 16Fw2 F' 15Lw2 15Dw2 F 9Uw' L' 15Fw2 9Uw2 15Rw2 R 14Bw2 14Lw2 14Uw2 B' 14Dw2 9Uw' 14Rw2 9Uw 14Lw2 13Bw2 13Rw2 13Bw2 13Lw2 F 13Rw2 9Uw' L 13Dw2 F' 13Rw2 R' 12Dw2 9Rw2 12Fw2 12Lw2 9Uw 12Rw2 B2 12Dw2 R' 12Fw2 9Bw2 11Lw2 R 11Dw2 F2 11Uw2 11Bw2 R' 9Uw B' 11Dw2 R 10Bw2 10Rw2 10Uw' F' 10Dw2 F' 10Fw2 10Uw' 9Rw2 10Bw2 10Lw2 10Uw2 B' 10Dw2 9Uw' 10Rw2 10Bw2 9Uw2 F R 10Dw2 R' 9Uw L' B' 9Uw' F' 9Dw' 8Uw' L' 16Dw2 F 8Dw' 16Lw2 8Uw' 8Fw2 R 16Bw2 15Uw2 L' 15Dw2 F' 15Lw2 8Uw L 15Dw2 R 15Fw2 8Rw2 14Bw2 14Uw2 14Fw2 8Uw' 14Uw2 14Bw2 L2 F' 14Dw2 L' 14Fw2 13Rw2 13Uw2 F 13Dw2 8Uw' 13Fw2 13Bw2 8Dw 13Fw2 R' 8Uw' 13Bw2 12Rw2 12Fw2 12Rw2 F 12Uw2 F' R' 12Bw2 8Uw 12Fw2 8Uw2 R' 11Dw2 11Rw2 F 11Rw2 8Uw 8Bw2 11Bw2 11Lw2 B2 10Uw2 10Lw2 10Dw2 F 10Lw2 10Fw2 8Dw F' R' 10Dw2 B2 9Dw B 9Rw2 9Dw L2 F 9Fw2 9Bw2 9Uw' 8Bw2 9Rw2 9Uw2 R 9Uw2 L' 9Dw2 8Uw' F' R' 8Bw2 8Uw' F2 16Dw2 16Rw2 B' 7Uw' 16Uw2 16Bw2 R 16Fw2 15Rw2 15Fw2 7Dw 15Lw2 R 15Uw2 R 15Dw2 B' 15Rw2 14Uw2 L2 14Dw2 7Uw' 14Lw2 R 14Dw2 7Lw2 F' 14Lw2 L' 13Dw2 13Bw2 13Rw2 7Uw' F' 13Dw2 L F' 13Dw2 B' 12Uw2 12Fw2 7Rw2 12Fw2 12Dw2 7Lw2 R' 7Dw' 12Lw2 B' 12Rw2 B 11Dw2 11Fw2 7Uw' 11Uw2 R' 11Fw2 11Lw2 B' R' 11Uw2 10Fw2 7Uw' 10Bw2 R2 7Uw R 10Uw2 7Rw2 10Dw2 7Uw 9Lw2 9Dw2 L 7Dw B 9Lw2 7Dw 9Bw2 R' 9Fw2 8Uw' L' 8Bw2 8Uw' F2 8Lw2 R' 8Rw2 8Dw' 8Rw2 F' 8Lw2 8Uw2 F2 8Dw2 7Uw 7Fw2 L 8Dw2 7Rw2 7Dw' B R 7Uw' 6Rw2 F R' 16Uw2 16Rw2 16Dw2 6Uw' 6Fw2 16Bw2 R' 16Dw2 15Rw2 15Dw2 15Bw2 15Rw2 15Dw2 6Fw2 L 6Uw' 15Rw2 15Fw2 15Bw2 14Dw2 B 14Uw2 14Bw2 14Lw2 6Dw' 14Fw2 L' 14Fw2 B' 6Uw' 14Lw2 F' 13Dw2 13Bw2 6Uw' 13Uw2 13Bw2 R 13Fw2 B' 6Uw' 13Bw2 R' 12Uw2 R' 6Uw B' 12Bw2 12Uw2 12Bw2 12Uw2 12Rw2 F' 12Uw2 6Rw2 11Uw2 L' 11Dw2 F' 6Uw' 11Uw2 L 11Fw2 11Rw2 6Dw 10Bw2 10Dw2 L 10Uw2 10Rw2 6Uw 10Rw2 6Uw 10Lw2 B' 10Rw2 9Fw2 6Dw' F' 9Lw2 9Uw2 9Bw2 R' 9Dw2 9Uw2 8Fw2 8Dw2 6Uw' L 8Fw2 8Lw2 B' 6Uw2 R 8Fw2 R' 8Uw2 R 7Uw' F B2 7Uw 7Bw2 7Dw2 F' 7Uw' 7Rw2 7Dw2 R 7Uw2 7Fw2 7Bw2 R' 6Dw 7Rw2 B' 7Uw2 6Bw2 6Uw' L' 6Dw' F 6Uw' F' 5Uw2 L' 16Dw2 B 16Lw2 R' 5Uw 16Rw2 15Rw2 15Fw2 5Uw' F' 15Rw2 15Uw2 15Bw2 R 5Dw' B' 15Rw2 R 14Dw2 F' 14Bw2 14Lw2 5Dw' R' 14Rw2 14Dw2 14Rw2 R' 5Uw' 13Rw2 13Uw2 13Rw2 13Bw2 R' 5Uw' F 13Dw2 5Uw 13Bw2 R' 12Uw2 12Lw2 5Dw 12Lw2 12Rw2 5Fw2 R' 5Uw' 12Uw2 F' 12Rw2 11Fw2 5Rw2 11Dw2 B 11Uw2 R 11Dw2 5Uw' 11Fw2 5Bw2 5Uw 11Rw2 R' 10Fw2 5Uw L 10Uw2 B 10Dw2 5Uw 10Rw2 B 5Uw 10Bw2 5Dw B 9Uw2 9Lw2 F' 9Bw2 9Lw2 9Bw2 5Dw2 L 9Bw2 5Uw 9Fw2 B 8Lw2 B' 5Dw' 8Dw2 R 8Uw2 F' 8Lw2 8Fw2 8Bw2 7Rw2 7Dw2 7Fw2 7Uw2 R' 5Uw' 7Rw2 5Dw2 B 7Dw2 6Rw2 B2 6Uw F 6Bw2 6Uw' F 6Lw2 R2 6Dw' B 6Rw2 6Fw2 R2 6Rw2 6Bw2 5Uw' 6Lw2 R' 6Dw2 R' 5Rw2 5Uw' L F 5Dw' 16Dw2 4Uw 16Lw2 4Uw 16Fw2 4Uw B' 16Rw2 4Uw' 16Lw2 R 15Uw2 15Fw2 4Rw2 15Uw2 15Lw2 F 15Rw2 4Uw R' 4Uw2 15Fw2 4Bw2 14Lw2 F' 14Rw2 14Bw2 L 14Uw2 B2 14Dw2 B 4Uw' 14Rw2 R 4Dw2 13Rw2 4Uw2 13Uw2 13Fw2 L' B 4Uw' 13Lw2 F 13Rw2 R' 12Bw2 L' F' 12Rw2 4Uw 12Lw2 R 12Dw2 12Bw2 11Rw2 B' 4Uw' 11Uw2 11Bw2 4Rw2 F 11Rw2 F 11Lw2 10Uw2 B' 10Dw2 10Rw2 10Fw2 10Bw2 L 4Fw2 4Uw 10Lw2 9Uw2 9Rw2 B' R' 9Fw2 4Dw R' 9Bw2 4Dw' R' 9Dw2 9Rw2 R2 8Dw2 4Uw2 B2 4Dw' 8Rw2 4Dw' F' 8Lw2 R 8Fw2 7Uw2 7Rw2 F R' 7Fw2 7Dw2 7Lw2 4Uw 4Fw2 R' 7Fw2 6Uw2 4Rw2 6Uw2 L' 4Dw 6Fw2 6Lw2 6Fw2 B 6Lw2 5Dw' L' F' 5Lw2 5Uw2 L' F 5Dw' L' 5Dw2 F 5Uw2 B' 4Uw 5Bw2 5Lw2 4Dw' L 5Fw2 4Rw2 F' R 4Bw2 4Uw' R' 3Uw 16Bw2 16Lw2 F' 16Fw2 3Uw' R 16Dw2 L' 16Fw2 15Fw2 3Dw 15Lw2 R' 15Fw2 3Lw2 B 15Lw2 R' 3Dw' F 14Bw2 14Uw2 14Rw2 3Uw2 14Rw2 3Dw' R' 14Dw2 3Dw' 13Fw2 B' 13Uw2 L' 13Dw2 L 13Lw2 3Dw F' 13Dw2 R 12Dw2 3Uw' F 12Uw2 12Fw2 L' 3Lw2 12Uw2 B 3Uw 12Fw2 11Fw2 11Dw2 B R' 11Dw2 3Fw2 3Dw' L 11Fw2 3Dw' B' 10Dw2 R 10Uw2 F2 3Uw L' 10Lw2 B 10Dw2 9Uw2 9Bw2 L' 9Fw2 B 9Uw2 3Rw2 3Fw2 9Dw2 3Uw' 9Fw2 3Rw2 3Uw 8Fw2 8Bw2 8Lw2 R2 8Uw2 L' 3Dw 8Rw2 B 8Lw2 3Bw2 3Uw 7Fw2 B' 7Bw2 7Dw2 L' 7Fw2 3Uw' 7Bw2 7Rw2 3Rw2 6Dw2 3Uw' 6Uw2 6Rw2 F 6Fw2 3Uw2 6Bw2 R 6Uw2 5Bw2 L' 5Lw2 5Uw2 B 5Dw2 3Uw' 3Lw2 5Rw2 F' 3Uw L' 5Dw2 R' 4Uw' 4Lw2 4Uw2 F 4Uw' F' 4Lw2 4Uw' 3Rw2 4Bw2 4Rw2 4Dw2 3Uw' L' 4Uw2 F 3Uw 4Rw2 3Uw 4Lw2 R' 3Uw2 B 3Rw2 3Uw' 2Rw2 16Bw2 2Uw B' 16Dw2 16Bw2 2Uw' R' 2Uw2 16Fw2 2Uw 16Rw2 B 15Dw2 15Lw2 15Bw2 2Dw' L 15Rw2 15Dw2 L' 2Rw2 2Uw' 15Rw2 14Rw2 14Bw2 14Dw2 14Rw2 B 14Uw2 R' 2Uw F' 2Uw 14Fw2 2Rw2 2Dw F' 13Uw2 2Lw2 13Fw2 B2 13Dw2 F 13Rw2 R' 2Dw B 12Bw2 2Uw' 12Fw2 12Dw2 R 12Bw2 L 12Uw2 B 11Dw2 11Fw2 11Lw2 R 11Uw2 2Rw2 2Uw2 F' 2Uw 11Fw2 R2 10Uw2 F2 10Dw2 R 2Dw 10Fw2 L' 10Lw2 B 10Bw2 2Uw' 10Rw2 9Rw2 2Uw' 9Bw2 9Uw2 2Fw2 B 9Dw2 L' 2Uw' 9Bw2 9Rw2 8Rw2 8Bw2 L 8Bw2 8Dw2 L F' B 2Uw 8Bw2 B' 7Lw2 7Uw2 L' 7Dw2 7Fw2 R' 7Fw2 2Rw2 7Fw2 B' 7Rw2 R 6Uw2 L 6Dw2 2Uw2 2Bw2 6Lw2 6Rw2 F' 2Uw' 6Lw2 5Rw2 5Uw2 5Rw2 5Dw2 F' 5Fw2 2Uw' 5Bw2 L' 2Dw B 5Rw2 4Bw2 4Rw2 4Dw2 4Lw2 4Dw2 F 2Dw' 4Bw2 4Lw2 B 2Uw' R' 4Bw2 R2 3Uw' 3Fw2 3Bw2 L 3Dw' L R2 3Dw' 2Rw2 3Uw2 R' 3Fw2 2Dw' 3Bw2 3Uw2 3Bw2 3Dw2 B' 3Dw2 2Dw' L' B 2Rw2 2Dw2 R' 2Uw B' 2Dw' 16Rw2 U' 15Rw2 15Bw2 16Bw2 16Lw2 16Fw2 U' 16Lw2 U' 16Uw2 L 16Uw2 16Fw2 L2 16Dw2 16Fw2 16Bw2 L 15Fw2 L' 15Bw2 15Rw2 B2 16Lw2 16Dw2 15Lw2 F 16Lw2 F' 16Uw2 F 15Lw2 F 16Rw2 15Dw2 U 16Bw2 U' 14Fw2 16Lw2 16Uw2 14Fw2 16Bw2 U' 14Lw2 16Bw2 L' 16Bw2 D2 16Fw2 16Dw2 16Bw2 D2 14Fw2 16Dw2 L' 14Bw2 F 14Uw2 B 16Lw2 16Uw2 14Lw2 B' 14Lw2 F' 14Dw2 16Rw2 16Dw2 F' 14Dw2 14Rw2 U' 14Fw2 15Lw2 U 15Lw2 15Bw2 D 14Rw2 14Uw2 L D2 14Fw2 15Fw2 14Bw2 L 15Dw2 15Fw2 14Bw2 14Uw2 B' 15Uw2 15Rw2 F' 15Dw2 14Rw2 B' 14Lw2 14Dw2 15Dw2 14Rw2 U2 15Lw2 16Uw2 16Bw2 13Rw2 U 13Bw2 13Lw2 13Fw2 U 13Rw2 16Uw2 13Fw2 16Uw2 L2 13Uw2 16Bw2 L' 13Dw2 L' 13Fw2 B2 16Uw2 F 13Lw2 F' 16Dw2 13Rw2 16Dw2 16Uw2 13Rw2 13Dw2 13Rw2 15Uw2 13Bw2 13Lw2 U' 15Rw2 13Bw2 15Rw2 15Bw2 13Lw2 L' U2 13Bw2 L2 13Fw2 15Uw2 L 15Bw2 15Dw2 13Bw2 B' 15Uw2 15Lw2 15Uw2 13Rw2 B2 13Rw2 F 13Dw2 F B 15Rw2 14Bw2 U 13Bw2 14Uw2 14Bw2 14Lw2 13Bw2 U' 14Fw2 L 14Bw2 L' 13Fw2 13Dw2 14Fw2 L' 13Uw2 L' 13Rw2 B2 13Uw2 14Lw2 B 13Dw2 14Rw2 D2 B 13Dw2 14Rw2 14Dw2 14Lw2 U2 13Rw2 16Rw2 U 16Fw2 U' 16Bw2 16Rw2 U 16Bw2 U 12Rw2 L 12Bw2 L 12Uw2 12Fw2 16Bw2 L' 12Bw2 16Dw2 12Fw2 B 12Lw2 12Uw2 B 16Uw2 F' 12Lw2 B' R2 16Uw2 R2 16Dw2 15Bw2 12Rw2 12Bw2 12Lw2 15Bw2 15Rw2 12Bw2 U' 12Fw2 D' L2 15Fw2 12Dw2 12Bw2 15Bw2 15Uw2 L 12Uw2 L' 12Fw2 B 12Rw2 F' 15Dw2 B' 12Rw2 12Uw2 F' 15Uw2 12Lw2 12Dw2 12Rw2 12Bw2 14Rw2 14Uw2 14Lw2 12Bw2 14Bw2 U' 14Bw2 U' 14Rw2 12Bw2 12Dw2 12Fw2 L2 U2 14Fw2 L' 14Fw2 14Uw2 L 14Fw2 R2 12Uw2 B 12Lw2 B 14Rw2 12Dw2 F2 12Lw2 14Lw2 14Dw2 12Lw2 14Rw2 12Uw2 13Bw2 12Lw2 U' 12Bw2 12Lw2 13Rw2 13Bw2 U 13Lw2 D 13Lw2 13Fw2 13Dw2 L' 13Uw2 13Fw2 L' 13Uw2 12Fw2 L2 12Uw2 L' 13Rw2 B' 12Rw2 B 13Uw2 F 12Rw2 F' 12Dw2 13Dw2 B2 13Lw2 13Uw2 12Lw2 13Rw2 16Fw2 11Lw2 U' 11Fw2 U 16Lw2 11Bw2 16Lw2 16Bw2 16Uw2 L' 16Fw2 11Bw2 D2 11Uw2 11Bw2 L' 16Fw2 16Rw2 B 11Lw2 F' 11Lw2 16Dw2 16Lw2 B 11Dw2 F B2 16Rw2 11Rw2 15Uw2 15Rw2 U' 11Bw2 11Lw2 11Fw2 D 15Fw2 11Lw2 15Bw2 11Dw2 15Bw2 L2 11Dw2 15Fw2 L' 11Uw2 L' 15Dw2 15Uw2 11Bw2 15Dw2 L2 11Dw2 B 15Rw2 11Uw2 15Lw2 B 11Dw2 U2 15Uw2 15Lw2 14Bw2 U 11Fw2 14Rw2 14Uw2 14Bw2 11Lw2 U' 11Bw2 14Dw2 L 11Dw2 11Uw2 11Fw2 L 11Uw2 14Bw2 L2 14Dw2 14Fw2 11Dw2 14Lw2 B' 11Uw2 F' 11Rw2 11Dw2 14Rw2 11Dw2 14Uw2 13Uw2 11Fw2 11Lw2 U 13Bw2 13Rw2 11Fw2 11Lw2 13Rw2 11Uw2 L 13Bw2 L 11Uw2 13Bw2 11Dw2 13Dw2 11Fw2 11Dw2 11Rw2 13Dw2 13Rw2 F' U2 13Rw2 11Uw2 13Rw2 F' 11Lw2 11Bw2 U 12Bw2 U 11Bw2 U 12Bw2 12Lw2 D 12Lw2 11Fw2 L2 11Bw2 L' 12Dw2 L 11Uw2 12Bw2 L 11Fw2 11Rw2 12Dw2 12Rw2 11Dw2 B' 12Lw2 11Uw2 11Lw2 11Rw2 11Dw2 B 12Uw2 11Rw2 12Rw2 12Dw2 16Uw2 10Rw2 U' 16Fw2 U' 10Rw2 10Fw2 10Bw2 U' 10Bw2 16Uw2 16Fw2 16Bw2 L' 10Fw2 L' 10Bw2 16Bw2 10Uw2 B2 10Rw2 16Dw2 B 10Uw2 16Uw2 10Rw2 16Uw2 L2 B' 10Lw2 16Dw2 15Rw2 15Bw2 10Rw2 U' 10Bw2 U 15Bw2 10Lw2 U' 10Fw2 D' L2 15Fw2 15Dw2 10Uw2 L 15Fw2 15Dw2 10Bw2 L 10Fw2 15Fw2 B2 10Uw2 L2 15Dw2 15Lw2 B' 10Uw2 B 10Lw2 B 10Dw2 F 15Rw2 14Bw2 10Rw2 14Bw2 14Lw2 U' 10Fw2 14Uw2 10Rw2 10Bw2 D' 14Bw2 L 14Fw2 L 14Fw2 14Bw2 10Uw2 L' 14Dw2 10Bw2 L 14Fw2 B' 10Uw2 F 14Rw2 10Dw2 L2 14Uw2 10Rw2 F 10Rw2 D2 10Uw2 F 14Rw2 U' 13Fw2 10Rw2 13Bw2 10Rw2 U 10Bw2 10Lw2 10Fw2 U' 13Bw2 U2 13Bw2 13Dw2 10Fw2 L' 10Uw2 L' 10Dw2 13Fw2 10Bw2 13Dw2 13Rw2 10Uw2 10Rw2 B 13Lw2 B 13Dw2 B' 10Dw2 U2 B' 13Rw2 10Fw2 10Lw2 D' 12Bw2 12Rw2 10Bw2 10Lw2 12Rw2 10Dw2 12Uw2 L2 10Uw2 L' 12Fw2 12Dw2 L 10Bw2 12Bw2 12Rw2 10Dw2 B 10Dw2 12Dw2 10Rw2 F2 10Lw2 U2 10Uw2 12Lw2 B' 10Dw2 12Rw2 11Bw2 11Uw2 11Rw2 10Fw2 U 10Bw2 11Rw2 10Fw2 D 11Fw2 11Lw2 10Bw2 11Dw2 10Fw2 L' 10Fw2 D2 11Fw2 L' 11Uw2 11Bw2 L' 10Lw2 11Dw2 10Rw2 F' 10Lw2 11Dw2 F B2 10Lw2 B 11Dw2 10Uw2 16Rw2 9Bw2 16Rw2 16Fw2 9Lw2 16Bw2 U' 9Fw2 9Bw2 U' 9Dw2 9Fw2 16Uw2 L' 9Fw2 16Bw2 L' 16Fw2 B2 9Lw2 B2 16Lw2 F2 R2 16Dw2 9Uw2 9Rw2 16Rw2 9Dw2 D 15Rw2 9Bw2 9Rw2 15Rw2 15Fw2 15Bw2 9Rw2 15Fw2 15Bw2 15Dw2 L' 9Fw2 L2 9Dw2 9Bw2 L' F2 9Dw2 F 15Rw2 B 9Uw2 B2 9Rw2 15Dw2 B2 15Lw2 14Fw2 9Rw2 9Fw2 14Fw2 14Lw2 9Fw2 U' 9Fw2 D' 14Bw2 9Uw2 L D2 14Dw2 14Uw2 14Fw2 L 9Dw2 L' 9Fw2 14Fw2 B2 14Lw2 D2 14Dw2 F2 14Lw2 9Rw2 B 9Uw2 B 14Dw2 B 9Lw2 13Fw2 13Bw2 U 9Rw2 9Fw2 9Bw2 U' 9Bw2 L2 13Dw2 9Fw2 L 9Uw2 9Fw2 L' 13Uw2 13Fw2 13Bw2 B2 9Rw2 B 9Dw2 9Uw2 9Lw2 B' 13Uw2 F 9Dw2 F' 13Dw2 9Rw2 12Bw2 U 12Rw2 9Fw2 9Bw2 12Rw2 9Fw2 U' 9Rw2 9Bw2 L' 9Dw2 9Uw2 9Fw2 L' 12Bw2 L2 12Fw2 12Bw2 L' R2 9Dw2 12Lw2 F' 9Lw2 B2 12Rw2 9Dw2 12Rw2 F' 12Lw2 U2 9Lw2 9Rw2 U' 11Rw2 U 11Bw2 9Rw2 11Bw2 D' 9Dw2 11Uw2 9Bw2 L' 11Uw2 L' 9Dw2 11Fw2 B 11Dw2 11Uw2 B' 9Rw2 11Rw2 11Uw2 F2 U2 11Rw2 11Dw2 9Uw2 F 9Lw2 10Rw2 9Bw2 9Rw2 U 9Rw2 U' 10Fw2 9Rw2 10Rw2 L2 10Dw2 L U2 9Fw2 10Dw2 10Bw2 L 10Uw2 10Fw2 10Rw2 B 9Lw2 10Rw2 B' 10Dw2 U2 9Rw2 F' 10Dw2 F2 B 9Dw2 10Dw2 U' 16Fw2 16Rw2 16Fw2 16Bw2 16Rw2 U' 8Bw2 U' 8Rw2 L 8Dw2 8Bw2 16Uw2 L 16Dw2 L 16Bw2 L' 16Dw2 L' 8Rw2 8Uw2 F' 16Uw2 8Rw2 16Rw2 16Dw2 U2 16Lw2 8Rw2 16Rw2 F' 8Rw2 16Rw2 15Rw2 15Uw2 15Fw2 8Rw2 U 8Rw2 15Fw2 D' 8Fw2 15Lw2 8Uw2 15Bw2 8Uw2 L' 15Bw2 15Dw2 8Fw2 8Uw2 L 15Bw2 8Rw2 B' 15Rw2 8Dw2 U2 15Rw2 F 8Uw2 15Rw2 F2 15Lw2 15Uw2 8Rw2 14Lw2 U 14Rw2 8Fw2 8Rw2 14Rw2 D 14Fw2 8Rw2 8Uw2 L' 8Fw2 8Dw2 8Uw2 L' 8Fw2 14Bw2 14Dw2 8Bw2 8Rw2 8Dw2 B2 14Dw2 B' 14Rw2 D2 8Dw2 14Rw2 F 8Rw2 14Dw2 8Uw2 13Rw2 U' 8Lw2 13Bw2 8Rw2 13Uw2 13Rw2 D' U2 8Fw2 D2 13Bw2 L' 8Dw2 L 13Dw2 8Fw2 B' R2 8Uw2 13Uw2 F2 8Uw2 13Rw2 B 13Uw2 B2 13Lw2 8Uw2 13Rw2 13Dw2 12Rw2 8Fw2 8Bw2 12Rw2 12Fw2 U 8Rw2 12Uw2 8Lw2 U' 8Uw2 8Fw2 L' 12Bw2 L 8Bw2 L' 12Dw2 12Fw2 L 12Bw2 12Rw2 F 8Rw2 12Dw2 8Rw2 F 8Dw2 12Dw2 F2 8Rw2 8Uw2 8Lw2 12Lw2 11Bw2 U' 8Bw2 U' 11Bw2 8Rw2 11Bw2 8Rw2 L 8Bw2 8Dw2 8Fw2 L 11Fw2 11Dw2 11Uw2 8Bw2 B' 8Rw2 B' 8Rw2 8Dw2 11Uw2 B' 11Lw2 D2 8Dw2 11Lw2 B 11Dw2 10Rw2 10Bw2 U' 8Rw2 8Bw2 10Lw2 10Rw2 8Bw2 U 10Rw2 8Dw2 8Uw2 10Fw2 10Uw2 8Bw2 L 8Dw2 L' 10Dw2 8Bw2 8Uw2 F 10Dw2 F' 8Lw2 F 8Lw2 B 10Lw2 F2 10Lw2 10Uw2 B2 8Dw2 8Rw2 9Fw2 8Bw2 U 9Rw2 9Fw2 8Bw2 9Uw2 9Rw2 U' 8Bw2 L 8Bw2 8Dw2 9Fw2 L' 9Dw2 9Fw2 8Uw2 8Fw2 L' 8Bw2 8Rw2 B2 9Dw2 B 9Rw2 8Dw2 F 9Uw2 L2 9Lw2 8Dw2 9Dw2 8Uw2 8Rw2 U 16Lw2 U 7Rw2 7Fw2 7Bw2 U' 16Lw2 16Dw2 L' 7Fw2 L' 16Dw2 7Fw2 L' 7Fw2 7Uw2 L' 16Rw2 7Dw2 7Rw2 U2 16Lw2 F' 7Dw2 7Uw2 L2 F' 16Dw2 U 15Rw2 7Bw2 U' 7Lw2 D 15Fw2 7Lw2 7Uw2 15Bw2 15Dw2 7Fw2 7Uw2 L 7Uw2 15Bw2 15Dw2 L 15Fw2 B2 15Uw2 B2 15Lw2 15Dw2 7Lw2 F' 7Rw2 7Uw2 7Lw2 B' 7Lw2 15Rw2 7Bw2 U 14Lw2 U' 7Lw2 14Lw2 14Uw2 14Bw2 7Lw2 D' L' 7Fw2 L 7Fw2 14Dw2 L' 7Bw2 L' 7Dw2 7Fw2 14Bw2 R2 14Dw2 B 7Rw2 B' 14Rw2 U2 14Uw2 7Rw2 7Uw2 14Uw2 14Rw2 13Rw2 U' 7Bw2 U' 13Rw2 U 7Rw2 D' 7Rw2 13Bw2 7Rw2 7Uw2 13Fw2 13Dw2 L 13Fw2 13Dw2 U2 13Fw2 13Bw2 L 13Fw2 7Rw2 B' 13Dw2 B2 13Uw2 7Lw2 13Lw2 D2 7Dw2 B 13Dw2 7Uw2 13Rw2 12Rw2 U' 12Fw2 7Rw2 7Fw2 12Uw2 12Fw2 12Rw2 D' L2 7Bw2 7Dw2 7Fw2 D2 7Bw2 12Bw2 L 12Uw2 L 12Fw2 7Uw2 B' 12Uw2 B' 12Lw2 F' 7Uw2 F' 7Dw2 L2 12Rw2 12Uw2 F' 7Dw2 11Rw2 7Bw2 U' 11Rw2 7Fw2 11Rw2 7Bw2 D' 7Lw2 L 11Bw2 7Dw2 7Bw2 11Bw2 11Uw2 7Bw2 L 7Fw2 L' 11Dw2 B 7Rw2 F 7Dw2 11Uw2 B 7Dw2 7Rw2 7Uw2 B' 7Rw2 7Dw2 11Dw2 10Bw2 U' 7Lw2 U' 7Rw2 10Fw2 7Rw2 10Bw2 7Uw2 L' 7Uw2 L 7Uw2 10Bw2 10Lw2 10Uw2 B 10Dw2 7Lw2 10Uw2 B' 7Rw2 7Dw2 10Lw2 10Uw2 7Rw2 9Bw2 U' 7Bw2 7Lw2 D 7Fw2 7Bw2 9Lw2 7Fw2 U2 9Bw2 9Uw2 9Fw2 7Dw2 L' 7Bw2 L 9Fw2 7Rw2 9Dw2 7Uw2 7Rw2 9Dw2 B2 9Uw2 F' 7Lw2 7Rw2 B 7Dw2 9Uw2 U' 8Lw2 8Fw2 7Rw2 8Fw2 U 8Bw2 8Lw2 D' 8Bw2 8Dw2 L U2 7Bw2 8Bw2 L 7Dw2 8Fw2 L 7Uw2 8Fw2 F 7Uw2 8Lw2 8Dw2 8Uw2 B' 7Uw2 B2 8Rw2 8Uw2 7Lw2 F 7Lw2 8Rw2 6Rw2 16Bw2 16Rw2 16Fw2 16Bw2 16Rw2 U' 6Bw2 U 16Fw2 D' 6Uw2 16Bw2 L' D2 U2 6Bw2 L 16Fw2 16Uw2 6Fw2 6Bw2 6Rw2 6Dw2 F' 6Lw2 6Dw2 6Uw2 B' 16Dw2 16Uw2 F' B' 16Uw2 6Lw2 15Fw2 U 6Lw2 15Uw2 6Rw2 U' L2 6Fw2 L' 15Fw2 6Bw2 L' 6Dw2 15Bw2 15Rw2 B 6Uw2 B' 6Uw2 15Rw2 6Uw2 6Rw2 15Dw2 U2 6Lw2 15Dw2 15Uw2 14Rw2 U 6Fw2 6Bw2 U' 14Rw2 D 14Fw2 14Bw2 6Lw2 14Fw2 L 6Dw2 14Uw2 L' 14Dw2 L' 14Fw2 6Bw2 6Uw2 L 14Bw2 14Uw2 6Lw2 14Dw2 F' 6Uw2 L2 B' 14Dw2 6Lw2 14Uw2 6Lw2 13Fw2 U' 6Rw2 6Fw2 13Uw2 6Lw2 U' L' 6Dw2 13Dw2 L2 6Dw2 6Fw2 L' 6Fw2 6Uw2 13Bw2 B2 13Uw2 6Rw2 F 13Uw2 L2 6Uw2 6Rw2 6Dw2 13Uw2 F' 13Dw2 6Rw2 12Fw2 12Uw2 12Rw2 6Bw2 U' 12Rw2 D' 12Lw2 12Dw2 12Uw2 L2 12Dw2 12Fw2 L' 6Bw2 6Uw2 6Fw2 L' B2 12Rw2 6Dw2 B 6Lw2 F' 12Dw2 B 6Dw2 B 12Uw2 12Lw2 6Uw2 U 11Bw2 U' 6Bw2 6Lw2 U' 6Bw2 L 6Dw2 L2 6Dw2 6Uw2 L' 11Bw2 11Uw2 B' 11Lw2 6Uw2 6Lw2 U2 11Lw2 B 6Uw2 L2 11Uw2 6Rw2 U 6Lw2 10Bw2 U' 6Rw2 U 6Lw2 6Bw2 L' 10Uw2 6Fw2 6Uw2 10Uw2 L D2 10Fw2 10Uw2 6Bw2 F 10Lw2 B 6Dw2 10Rw2 6Uw2 10Rw2 F' R2 10Dw2 F 6Uw2 9Rw2 9Fw2 6Lw2 U 9Rw2 6Bw2 6Rw2 9Rw2 D' 9Lw2 9Uw2 6Bw2 L 9Fw2 9Uw2 6Fw2 9Dw2 L 6Uw2 9Bw2 B' 6Uw2 F 9Rw2 D2 9Lw2 6Dw2 L2 9Dw2 F 6Uw2 F 9Lw2 6Rw2 8Rw2 U' 8Rw2 6Fw2 6Rw2 8Rw2 D 8Fw2 6Rw2 8Bw2 6Dw2 6Bw2 8Uw2 L 8Bw2 8Dw2 6Uw2 8Bw2 L 6Bw2 8Bw2 F' 6Uw2 B' 8Rw2 6Uw2 B' 6Uw2 B 8Dw2 6Rw2 6Dw2 F2 8Rw2 6Rw2 U 6Lw2 7Rw2 7Bw2 U 6Bw2 7Bw2 7Rw2 7Bw2 6Lw2 6Bw2 L2 6Uw2 L' 6Dw2 L2 6Bw2 L' 7Dw2 6Fw2 7Bw2 7Rw2 6Uw2 B 7Lw2 7Uw2 F2 6Dw2 7Lw2 6Uw2 B' 7Lw2 6Dw2 B' 6Uw2 16Rw2 U 5Bw2 5Lw2 U 16Rw2 U 5Bw2 D' 5Bw2 16Dw2 16Uw2 L' 16Bw2 L 5Uw2 L' U2 16Uw2 16Bw2 L' 5Rw2 16Dw2 5Uw2 5Lw2 16Lw2 B 5Dw2 F' U2 B2 5Uw2 16Uw2 16Rw2 U' 15Fw2 5Lw2 15Fw2 15Rw2 U' 5Bw2 D' 5Dw2 5Uw2 5Bw2 L 5Fw2 L 15Uw2 5Bw2 L2 15Bw2 15Rw2 F 5Dw2 B' 5Rw2 15Dw2 15Rw2 F2 15Dw2 F' R2 5Uw2 5Bw2 14Uw2 14Rw2 U 5Rw2 D 14Bw2 5Lw2 14Uw2 L2 5Bw2 L 5Bw2 L' D2 5Dw2 14Bw2 14Rw2 14Dw2 F2 14Dw2 F 5Uw2 F' 5Rw2 5Dw2 14Uw2 14Rw2 5Dw2 14Rw2 5Rw2 13Bw2 U 5Fw2 5Bw2 U' 13Fw2 5Rw2 13Uw2 5Fw2 13Dw2 5Uw2 L' 5Bw2 13Dw2 L' 5Uw2 L' 13Fw2 13Bw2 5Uw2 13Lw2 5Dw2 5Rw2 13Uw2 B 5Lw2 F' 5Lw2 D2 5Uw2 13Lw2 13Dw2 5Bw2 U' 12Fw2 U 12Rw2 5Bw2 12Rw2 5Fw2 U' 12Fw2 5Bw2 L2 12Uw2 5Bw2 5Dw2 L' 5Fw2 L' R2 12Dw2 12Rw2 B 5Uw2 5Lw2 F2 B' 12Lw2 5Uw2 12Rw2 5Uw2 U 11Bw2 5Rw2 11Bw2 11Lw2 U 5Lw2 D' 11Uw2 L2 D2 11Fw2 L' 11Dw2 5Uw2 5Fw2 11Dw2 L' 5Rw2 5Uw2 B2 11Lw2 F 5Rw2 F' B2 11Dw2 11Lw2 11Rw2 5Uw2 11Rw2 11Uw2 5Rw2 10Bw2 10Rw2 5Fw2 10Rw2 U' 5Lw2 U' 10Dw2 L' 10Dw2 L' 5Bw2 5Uw2 10Bw2 10Dw2 5Bw2 5Uw2 B' 10Rw2 F 10Uw2 F 5Rw2 5Uw2 F U2 5Uw2 10Rw2 9Bw2 9Uw2 9Rw2 5Fw2 U 9Lw2 U' 5Fw2 9Rw2 9Uw2 9Fw2 L2 9Fw2 L2 9Uw2 5Bw2 9Bw2 5Dw2 9Fw2 9Rw2 5Uw2 B' 9Uw2 5Rw2 9Uw2 F' R2 9Uw2 9Rw2 5Uw2 F2 9Dw2 5Lw2 8Bw2 5Rw2 8Bw2 5Rw2 U' 8Lw2 5Bw2 8Lw2 L2 8Bw2 L' 5Bw2 8Bw2 5Uw2 8Bw2 L' 8Bw2 8Rw2 B' 5Lw2 F2 5Uw2 8Rw2 8Uw2 8Rw2 F' 8Dw2 5Rw2 5Dw2 8Uw2 5Rw2 7Uw2 5Fw2 D' 7Bw2 7Lw2 5Bw2 5Lw2 7Lw2 L' 7Bw2 5Uw2 7Bw2 5Dw2 L' D2 5Fw2 5Bw2 7Bw2 5Rw2 B2 5Lw2 7Rw2 U2 7Uw2 B2 7Rw2 B' 5Lw2 5Rw2 B 5Dw2 7Uw2 U 6Lw2 6Fw2 5Bw2 6Lw2 U' 5Fw2 6Uw2 6Fw2 U' 5Uw2 L2 D2 6Dw2 5Bw2 6Uw2 6Fw2 L 6Uw2 5Fw2 L' B' 6Lw2 5Dw2 5Lw2 6Rw2 6Dw2 6Rw2 B 5Uw2 6Rw2 F' 6Lw2 5Dw2 16Rw2 U 4Lw2 16Bw2 16Lw2 4Bw2 4Lw2 16Lw2 16Bw2 D' 4Dw2 16Bw2 L 4Uw2 L 16Bw2 L 16Dw2 4Fw2 16Rw2 F 4Rw2 B 16Rw2 4Dw2 B2 4Uw2 F 4Lw2 4Dw2 F' 16Uw2 15Fw2 U 4Rw2 15Fw2 4Lw2 15Uw2 4Fw2 15Rw2 U' L 15Uw2 L 4Uw2 15Fw2 4Dw2 L 15Bw2 L' 4Bw2 B 4Uw2 15Lw2 15Uw2 B 15Rw2 F' 4Uw2 15Uw2 F 4Dw2 15Rw2 4Uw2 14Lw2 U' 4Lw2 U 4Fw2 4Bw2 14Lw2 14Dw2 L' 4Dw2 L2 4Dw2 4Bw2 L' 4Fw2 F 4Dw2 4Lw2 4Uw2 F' 14Uw2 F 4Uw2 B 14Uw2 4Lw2 14Dw2 4Bw2 U' 13Rw2 13Bw2 U' 4Bw2 D 13Lw2 13Fw2 4Lw2 4Bw2 4Uw2 13Bw2 L 4Fw2 13Uw2 L' 13Fw2 4Uw2 13Fw2 L' B2 13Rw2 4Dw2 4Uw2 F2 13Dw2 B 4Uw2 4Rw2 B 13Rw2 F2 4Rw2 12Bw2 12Rw2 12Uw2 12Fw2 4Lw2 U' 4Bw2 U' 4Fw2 12Rw2 4Uw2 12Bw2 12Uw2 L2 4Fw2 12Bw2 12Dw2 L' 4Bw2 4Dw2 L' 4Rw2 4Dw2 4Uw2 F2 4Lw2 12Uw2 F' 4Uw2 B R2 4Dw2 12Uw2 4Rw2 11Bw2 U 4Fw2 4Bw2 U' 11Fw2 4Rw2 4Uw2 4Fw2 11Dw2 11Uw2 L' 11Bw2 U2 4Uw2 11Fw2 4Bw2 11Rw2 B' 11Lw2 4Dw2 11Lw2 D2 4Dw2 B 11Rw2 11Dw2 11Uw2 10Lw2 4Bw2 10Lw2 10Fw2 U' 4Rw2 4Fw2 4Lw2 U' 4Bw2 10Bw2 10Uw2 L' 4Bw2 4Uw2 10Bw2 L 4Dw2 L' 10Bw2 L' 4Rw2 F2 10Dw2 10Lw2 4Uw2 10Lw2 F' 4Uw2 F' 10Lw2 B2 4Lw2 9Rw2 4Bw2 U 9Rw2 4Fw2 9Rw2 U' 9Rw2 9Bw2 U2 4Uw2 4Bw2 9Bw2 9Uw2 4Fw2 L2 4Fw2 R2 9Uw2 B' 4Dw2 4Rw2 F 4Rw2 9Uw2 F2 9Dw2 8Bw2 U 4Rw2 U' 8Lw2 4Bw2 U 4Bw2 U' 8Lw2 L 8Dw2 4Uw2 4Fw2 8Dw2 L D2 8Fw2 8Dw2 4Lw2 8Dw2 4Uw2 B 4Dw2 4Lw2 4Rw2 B' 4Dw2 4Rw2 8Dw2 7Rw2 U' 4Bw2 4Lw2 4Fw2 D' 4Lw2 7Uw2 7Bw2 L' 7Fw2 4Uw2 7Fw2 L U2 7Bw2 7Dw2 4Bw2 B2 7Dw2 F B' 4Uw2 7Rw2 B' 7Uw2 4Rw2 7Uw2 F 7Rw2 4Uw2 6Rw2 U 4Rw2 D' 6Bw2 4Uw2 L' 4Uw2 6Bw2 4Uw2 6Bw2 L2 4Bw2 L' 4Dw2 6Bw2 6Rw2 B 4Rw2 4Uw2 B' 6Rw2 B' 4Uw2 B' 6Rw2 4Dw2 F' 4Lw2 5Lw2 4Rw2 U' 4Fw2 5Rw2 5Bw2 5Rw2 5Uw2 5Bw2 D' 4Bw2 5Dw2 4Uw2 L 4Bw2 5Uw2 5Fw2 4Uw2 L' 4Uw2 4Fw2 5Rw2 B 4Dw2 4Lw2 5Lw2 F 4Dw2 B 5Uw2 R2 5Dw2 5Lw2 B' 4Uw2 3Rw2 U 16Bw2 U 3Fw2 U' 16Fw2 3Rw2 L2 3Fw2 3Uw2 16Fw2 16Bw2 L 16Uw2 3Fw2 16Uw2 3Fw2 B2 16Dw2 3Rw2 F' 3Lw2 3Dw2 F' 3Lw2 16Dw2 B' 16Dw2 3Rw2 16Dw2 15Uw2 15Rw2 3Fw2 U 3Rw2 3Fw2 3Bw2 D 3Lw2 3Bw2 3Dw2 L 3Fw2 15Uw2 L2 U2 15Fw2 L' 3Uw2 R2 3Dw2 15Rw2 F' 3Uw2 B2 15Rw2 3Uw2 15Rw2 F B2 15Uw2 U' 14Bw2 3Lw2 14Bw2 14Rw2 U 3Fw2 D' L 3Dw2 L' 3Bw2 3Dw2 14Fw2 14Uw2 3Bw2 L' 3Uw2 L' 14Rw2 14Uw2 B2 14Uw2 3Lw2 F' 3Lw2 14Dw2 F 3Lw2 3Uw2 B' 14Lw2 13Lw2 U' 13Bw2 3Lw2 13Bw2 3Lw2 3Fw2 D' 3Uw2 3Fw2 3Dw2 L' 3Dw2 3Fw2 13Fw2 3Dw2 13Fw2 L 13Bw2 3Rw2 F2 13Lw2 B 13Lw2 3Rw2 3Dw2 13Lw2 B 13Lw2 F2 3Uw2 3Rw2 U 12Lw2 U 3Rw2 3Fw2 D' 12Fw2 3Lw2 12Uw2 L2 3Bw2 L 12Fw2 3Dw2 L' U2 12Fw2 12Dw2 3Bw2 3Dw2 12Dw2 B' 12Uw2 12Lw2 3Uw2 3Lw2 12Rw2 12Dw2 F' 12Rw2 11Bw2 U 3Bw2 3Lw2 3Fw2 U 3Lw2 11Bw2 L2 3Uw2 11Uw2 11Bw2 L 3Uw2 L' 3Fw2 11Bw2 3Uw2 3Rw2 F 3Rw2 F2 11Lw2 D2 3Dw2 11Lw2 F' 11Uw2 3Bw2 10Rw2 3Bw2 U' 10Fw2 3Rw2 10Fw2 D' 10Dw2 L' 10Dw2 10Fw2 L' 3Dw2 10Bw2 10Rw2 F2 10Rw2 10Dw2 3Rw2 B' 3Uw2 10Uw2 3Rw2 10Uw2 F2 B' 10Uw2 U 9Bw2 3Lw2 9Bw2 9Rw2 U 3Rw2 3Fw2 3Lw2 D' L 9Bw2 3Dw2 9Bw2 U2 3Fw2 L' D2 3Fw2 9Bw2 3Dw2 3Uw2 F' 9Rw2 3Uw2 9Rw2 D2 3Rw2 F 9Lw2 F 3Rw2 3Rw2 U' 3Bw2 U 8Fw2 8Bw2 3Rw2 3Uw2 8Uw2 L2 8Uw2 8Bw2 U2 3Fw2 L 8Uw2 L' 8Fw2 3Lw2 B' 3Lw2 3Uw2 B' 8Lw2 3Uw2 F B' 8Uw2 8Lw2 7Rw2 U' 3Bw2 U 7Fw2 3Rw2 U' 3Rw2 D 7Rw2 3Bw2 L2 3Uw2 7Bw2 L 3Fw2 7Bw2 3Uw2 7Bw2 7Rw2 3Dw2 F' 3Lw2 3Uw2 7Rw2 3Uw2 7Rw2 F 7Lw2 6Rw2 3Fw2 3Bw2 U 3Lw2 U' 6Rw2 3Fw2 L2 3Dw2 6Uw2 L 3Dw2 3Uw2 3Bw2 L' 3Dw2 6Dw2 6Lw2 B' 6Dw2 6Rw2 B' 3Rw2 3Uw2 6Uw2 3Rw2 3Dw2 F2 6Lw2 3Bw2 3Lw2 5Rw2 U 5Bw2 3Rw2 3Bw2 5Bw2 D' 5Bw2 5Uw2 L' 3Uw2 3Bw2 5Dw2 3Bw2 5Bw2 L' 5Fw2 5Rw2 5Uw2 B2 3Dw2 5Uw2 5Rw2 B' 3Uw2 F 5Lw2 5Uw2 F B2 3Dw2 3Rw2 4Fw2 3Rw2 3Fw2 U' 4Lw2 4Bw2 U' 3Fw2 4Lw2 L2 4Uw2 4Bw2 4Uw2 3Fw2 3Dw2 4Bw2 4Uw2 L 3Bw2 L' 4Rw2 4Uw2 3Rw2 4Uw2 3Rw2 B 4Rw2 4Dw2 L2 3Uw2 B' 3Lw2 F' 3Dw2 4Dw2 16Bw2 16Rw2 2Fw2 U 16Fw2 2Bw2 16Bw2 U' 16Bw2 16Rw2 16Dw2 2Fw2 L 16Dw2 L2 2Uw2 16Bw2 L2 2Fw2 L' 16Bw2 B2 2Uw2 2Rw2 2Dw2 16Uw2 16Rw2 16Dw2 2Uw2 2Lw2 F 2Lw2 2Rw2 B 16Lw2 15Bw2 U' 15Bw2 2Lw2 2Bw2 15Uw2 2Bw2 15Rw2 U' 15Bw2 2Uw2 15Bw2 2Dw2 15Uw2 L' 15Dw2 2Bw2 L2 2Dw2 15Fw2 L' 15Rw2 15Dw2 2Rw2 2Uw2 15Uw2 F' 2Uw2 B' 2Rw2 15Uw2 2Rw2 15Dw2 15Rw2 U2 2Rw2 14Lw2 U' 2Rw2 2Bw2 14Rw2 14Uw2 2Fw2 14Lw2 14Bw2 D' D2 14Bw2 14Dw2 2Fw2 L' 14Uw2 2Bw2 14Uw2 2Bw2 L' 14Rw2 B' R2 2Dw2 F 2Dw2 14Uw2 B' 2Rw2 U2 B' 14Rw2 13Fw2 2Rw2 13Bw2 2Rw2 U 2Rw2 2Bw2 2Lw2 U' 2Dw2 13Fw2 L 2Uw2 L 13Bw2 L' 2Dw2 L 13Fw2 R2 13Uw2 2Lw2 2Rw2 13Uw2 F' 13Uw2 B 2Lw2 2Rw2 B' 2Uw2 13Uw2 12Bw2 2Rw2 12Fw2 U' 2Rw2 2Bw2 2Lw2 U' 12Bw2 12Uw2 12Bw2 12Dw2 L 2Dw2 12Fw2 12Dw2 2Bw2 12Bw2 2Uw2 L' 12Lw2 F2 12Dw2 2Lw2 B2 2Uw2 12Rw2 B' 2Uw2 L2 B' 12Dw2 2Uw2 11Rw2 2Bw2 11Rw2 11Uw2 2Lw2 11Bw2 2Lw2 2Bw2 L 2Dw2 U2 11Bw2 11Dw2 11Fw2 L' 2Fw2 L2 11Bw2 B2 2Uw2 11Lw2 B2 2Rw2 F 2Rw2 2Uw2 2Lw2 11Rw2 B2 11Dw2 F 11Rw2 10Rw2 10Fw2 10Bw2 2Rw2 10Bw2 U 2Fw2 D' 10Lw2 2Dw2 U2 10Fw2 10Bw2 10Dw2 L 10Dw2 L 10Fw2 2Bw2 2Rw2 2Dw2 10Lw2 U2 B' 10Rw2 F2 10Dw2 2Uw2 B' 2Dw2 10Lw2 2Rw2 U' 9Bw2 U' 2Lw2 9Uw2 2Rw2 2Bw2 U' 9Bw2 9Uw2 9Bw2 2Uw2 L' 9Bw2 9Dw2 9Uw2 L U2 9Fw2 9Rw2 2Uw2 2Rw2 U2 2Lw2 9Rw2 9Dw2 9Uw2 F 9Dw2 2Rw2 9Dw2 2Rw2 U 8Rw2 2Fw2 8Rw2 8Bw2 U' 2Rw2 2Bw2 2Lw2 U' 8Fw2 2Bw2 8Uw2 L 8Uw2 8Fw2 2Dw2 8Fw2 2Dw2 2Fw2 8Uw2 L' F 2Dw2 8Rw2 2Uw2 8Lw2 2Rw2 B 8Rw2 F2 8Dw2 8Lw2 2Dw2 U' 7Fw2 2Rw2 7Fw2 7Lw2 U' 2Lw2 D' 2Uw2 2Bw2 U2 7Fw2 7Uw2 2Fw2 L' 2Bw2 L' 7Bw2 7Uw2 F 2Dw2 F' 2Lw2 7Rw2 F2 7Rw2 7Uw2 F2 2Dw2 2Rw2 2Rw2 U 6Bw2 U 2Fw2 2Lw2 U' 6Fw2 2Rw2 2Fw2 6Fw2 L 6Dw2 2Fw2 6Fw2 L' 2Uw2 L 6Fw2 F 2Uw2 2Rw2 2Dw2 F 6Dw2 B 6Dw2 2Rw2 6Dw2 F 2Rw2 5Rw2 5Bw2 5Uw2 2Fw2 U' 2Bw2 D' 2Bw2 5Rw2 5Bw2 5Uw2 5Bw2 5Dw2 2Uw2 L' 2Uw2 L' 2Dw2 5Fw2 5Dw2 2Bw2 5Dw2 5Uw2 2Rw2 5Uw2 B' 2Rw2 F' 5Dw2 2Lw2 5Uw2 B' 2Dw2 4Rw2 4Bw2 U' 2Rw2 D' 4Bw2 2Rw2 4Uw2 2Bw2 4Rw2 2Dw2 L 2Uw2 2Fw2 4Fw2 4Uw2 4Bw2 2Uw2 L' 4Dw2 4Bw2 2Dw2 2Uw2 4Rw2 F' 2Uw2 F' 2Dw2 4Uw2 F B' 4Uw2 4Lw2 2Uw2 U' 3Bw2 3Rw2 3Uw2 2Fw2 U' 2Bw2 D' 2Bw2 3Rw2 2Dw2 2Uw2 3Fw2 L' 3Dw2 3Uw2 L2 3Fw2 2Bw2 L 3Bw2 2Rw2 F2 2Rw2 2Uw2 3Rw2 B' 2Uw2 B2 2Uw2 F' 3Rw2 B' 2Uw2 16Lw2 D' F2 16Dw2 F 16Dw2 R U' 16Fw2 R' L2 16Rw2 B 16Rw2 D2 16Rw2 B 16Uw2 16Bw2 U2 B 16Rw2 B2 16Dw2 16Fw2 U2 R 16Fw2 U2 R2 16Bw2 U2 R' 16Dw2 F2 16Bw2 16Dw2 D' F U' 15Lw2 B 15Lw2 U' B R F2 15Dw2 15Fw2 15Dw2 R2 15Dw2 F' L2 15Rw2 F' 15Rw2 R F2 15Dw2 L B2 R 15Fw2 15Uw2 B2 R D2 15Bw2 15Dw2 15Bw2 15Dw2 F 14Dw2 U R' 14Fw2 U' F2 D' F L' F' D2 14Dw2 B' 14Lw2 14Uw2 F2 B R2 U2 F U2 14Lw2 F2 R B2 R F2 14Bw2 R' 14Uw2 14Fw2 U2 R2 14Dw2 L' 14Uw2 14Fw2 U F' 13Dw2 R B2 13Uw2 F D' 13Bw2 L' F 13Lw2 B' 13Lw2 R2 D2 13Uw2 B2 13Uw2 13Lw2 B 13Lw2 F2 13Fw2 R2 13Bw2 R2 13Dw2 L 13Uw2 13Fw2 D2 L 13Fw2 R 13Dw2 B R' 12Fw2 D 12Uw2 L2 R U' B R' 12Uw2 F2 12Lw2 U2 B' 12Lw2 F' 12Dw2 12Uw2 F2 L2 B 12Lw2 12Fw2 R2 F2 L 12Fw2 B2 12Dw2 R 12Fw2 B2 D2 R 12Uw2 12Bw2 R 12Uw2 B L2 U 11Bw2 U R2 F U' D2 L' 11Uw2 B U2 B' 11Dw2 U2 11Lw2 11Rw2 B 11Lw2 11Rw2 11Fw2 R F2 R 11Bw2 R2 D2 11Fw2 R2 11Dw2 U2 11Uw2 R 11Dw2 L2 11Dw2 D F' 10Uw2 R2 10Dw2 L U' B' 10Lw2 10Dw2 10Uw2 B2 10Lw2 R2 10Dw2 F' 10Uw2 10Lw2 F 10Rw2 10Dw2 R2 D2 10Bw2 L 10Fw2 R B2 L 10Bw2 L B2 L D2 10Uw2 10Bw2 9Bw2 D L' 9Uw2 F' 9Dw2 F' D' D2 L' L2 F L2 9Uw2 R2 9Dw2 9Rw2 F' 9Dw2 B 9Uw2 9Lw2 B' 9Dw2 9Fw2 L2 9Dw2 F2 L 9Dw2 L B2 L' D2 9Uw2 R F2 9Dw2 D' 8Dw2 F 8Lw2 B L 8Lw2 D' F' L' B L2 8Dw2 F' R2 B' D2 8Dw2 8Rw2 F' 8Rw2 F 8Bw2 D2 8Fw2 L' 8Dw2 F2 L 8Bw2 U2 B2 8Bw2 R 8Dw2 8Uw2 R2 8Fw2 B U' F' 7Dw2 B D' D2 L' 7Lw2 7Uw2 7Lw2 B' 7Rw2 7Dw2 B D2 7Lw2 B U2 7Lw2 7Dw2 R F2 L U2 7Bw2 7Dw2 U2 L 7Fw2 R' U2 L' F2 B2 7Dw2 6Lw2 6Rw2 D' R 6Bw2 L 6Bw2 D' L' R 6Uw2 B' L2 B' 6Rw2 U2 R2 F' U2 L2 F 6Lw2 6Fw2 L 6Uw2 B2 6Dw2 F2 6Fw2 L' D2 6Uw2 6Fw2 L' D2 U2 6Fw2 5Bw2 U' L' 5Dw2 R B2 U' 5Uw2 R' 5Uw2 R2 B 5Dw2 L2 5Dw2 U2 F' D2 5Rw2 B2 5Rw2 F' 5Bw2 D2 5Dw2 R 5Dw2 5Uw2 5Fw2 R U2 R' 5Uw2 5Fw2 5Uw2 B2 R' F 4Uw2 L 4Dw2 L D' D2 R F 4Lw2 F' 4Rw2 U2 4Rw2 U2 F 4Uw2 4Lw2 F' R2 4Dw2 L B2 R' 4Bw2 U2 L 4Bw2 4Uw2 R2 D2 R 4Bw2 4Uw2 R' 4Fw2 3Bw2 3Rw2 F D' 3Bw2 R' 3Fw2 D' B L' 3Dw2 U2 3Lw2 F2 L2 F' 3Lw2 3Dw2 3Fw2 L2 F 3Lw2 D2 3Fw2 D2 R 3Dw2 3Bw2 L 3Bw2 R' 3Uw2 F2 U2 3Fw2 R 3Uw2 F2 2Bw2 U 2Rw2 2Dw2 R' D' B R 2Fw2 2Dw2 F' R2 F' R2 F' R2 2Bw2 2Dw2 B 2Lw2 D2 2Rw2 B2 D2 2Bw2 2Uw2 R 2Uw2 F2 R' 2Bw2 2Uw2 F2 2Uw2 L 2Fw2 F' U2 B2 R' D' R B' U' F D2 B L2 F2 U2 B2 U2 L2 U2 R2 D2"

# subproblem

In [ ]:
subproblems=[]

In [408]:
import subprocess

normal_state=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[0]["solution_state"]
for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    if row["solution_state"]==normal_state:
        continue
    id = row['id']
    cur_state = row['initial_state']
    print("-"*30)
    print(id)

    input_state=simulate_inverse(
        normal_state, ## 普通の解
        row["moves"],
        row["allowed_moves"],
    )
    input_state=state2ubl(input_state)
    
    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
    sol=output.replace("Solution: ","").replace("\n","")
    print("sol", sol)
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    cur_state=simulate(
        row["initial_state"],
        mmoves,
        row["allowed_moves"],
    )
    subproblems.append({
        "id": id*1000,
        "puzzle_type": row["puzzle_type"],
        "moves": mmoves,
        "initial_state": cur_state,
        "solution_state": row["solution_state"],
        "num_wildcards": row["num_wildcards"]
    })

------------------------------
235
sol Uw' Bw L' Uw2 Bw' Lw2 Fw2 Dw Lw B Uw' U' Bw2 R' Uw2 R2 Fw' Uw2 L Bw' D2 Uw2 B U2 Rw2 Fw2 L' F' L D' Fw2 R' D2 L D2 Fw2 D2 R Fw2 L2 Dw2 B2 R' Uw2 Bw2 D R2 D' Rw2 U2 Bw2 Lw2 F2 Fw2 Rw2 D' L2 Rw2 D' Lw2 D2 U' R D B U' R D2 R' F' R' B' R2 B2 U2 F2 U2 B2 U L2 F2 U
------------------------------
236
sol Uw2 Rw' Lw2 B2 Bw2 Dw2 Uw Lw B Fw Dw' Fw2 U' Rw' Lw2 F' Lw' D Fw2 D2 Uw2 Lw' F U2 B L B' Fw2 L' D' F' Fw2 L2 U F2 Rw2 Lw2 U' L2 U' Rw2 U Fw2 Dw2 Fw2 F2 Rw2 F Lw2 B R2 Dw2 F' U2 Lw2 Uw2 R2 U2 B2 Uw2 F Lw2 L2 B D R' U2 B' U R F U' L' B R2 B' U2 L2 R2 F D2 B2 R2
------------------------------
237
sol Dw' Lw F' Uw' Fw' B' Rw' Fw2 L Bw' Fw2 R' Lw2 Uw' F' Rw2 Dw' B L Uw' B R2 U' L' Lw2 Dw2 R' D' B' R' D2 F' Lw2 U2 Dw2 R2 Fw2 L2 Dw2 Rw2 Uw2 F Rw2 F L' F2 L' D2 R2 Uw2 F2 Fw2 Dw2 Bw2 L2 B2 U2 R' Dw2 R D' L2 F U' B2 R2 L U2 B L2 B U2 R2 B2 D2 L U2 L U2 B2 R2
------------------------------
238
sol L' Uw' Fw2 Dw2 Rw D' Dw' Fw' L Lw2 Bw' Uw' R' Uw' R2 Dw Uw2 B' Rw2 U

In [411]:
subproblem_df=pd.DataFrame(subproblems).sort_values("id")
subproblem_df.to_csv("subproblem.csv", index=None)

In [412]:
subproblem_df

,id,puzzle_type,moves,initial_state,solution_state,num_wildcards
4,200000,cube_4/4/4,d0.-r0.-r1.d0.-f3.-f2.r0.d3.d2.-r3.-r3.f3.-f3....,A;B;A;B;A;A;A;B;A;B;B;B;A;B;A;B;B;C;B;C;B;C;B;...,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;...,0
5,201000,cube_4/4/4,d0.d1.d0.d1.-r3.-r3.f3.f2.-r3.-r3.-d0.-d1.f3.f...,A;B;A;B;A;A;A;B;A;B;B;B;A;B;A;B;B;C;B;C;B;C;C;...,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;...,0
6,202000,cube_4/4/4,-d0.-f0.-d0.-d1.f3.d3.d2.f3.f2.f3.-r0.-r1.-d3....,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;B;B;...,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;...,0
7,203000,cube_4/4/4,-d0.-d1.-f3.-f3.d0.r3.d0.d1.f3.f2.-d3.f0.-f3.-...,A;B;A;B;A;B;B;B;A;A;A;B;A;B;A;B;B;C;B;C;B;C;C;...,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;...,10
8,204000,cube_4/4/4,d0.d1.d0.d1.r0.r1.f3.f2.r3.-f3.d3.d2.-d3.-d3.d...,A;B;A;B;A;A;B;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;C;...,A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;...,0
9,205000,cube_4/4/4,f3.f2.d0.d1.r0.r1.d3.d2.-r3.-r3.f3.f2.-d3.-d3....,N0;N1;N2;N3;N4;N9;N5;N7;N8;N6;N10;N11;N12;N13;...,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,0
10,206000,cube_4/4/4,d0.d1.-f3.-f2.-f3.-f2.r0.r0.-d3.-d3.-r3.-r3.d3...,N0;N1;N2;N3;N4;N10;N5;N7;N8;N6;N9;N11;N12;N13;...,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,10
11,207000,cube_4/4/4,-r0.-r1.-d0.-r3.d0.d1.d0.d1.-r0.-f0.-f1.r0.r0....,N0;N1;N2;N3;N4;N9;N5;N7;N8;N10;N6;N11;N12;N13;...,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,0
12,208000,cube_4/4/4,d0.d1.d0.d1.f3.f2.-d3.-d3.-r0.-r1.-f3.-f3.d3.d...,N0;N1;N2;N3;N4;N6;N10;N7;N8;N5;N9;N11;N12;N13;...,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,6
13,209000,cube_4/4/4,d0.d1.d0.d1.-r0.-r1.-f3.-f2.-d0.f3.f2.r0.r0.-d...,N0;N1;N2;N3;N4;N5;N10;N7;N8;N6;N9;N11;N12;N13;...,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,0
